<a href="https://colab.research.google.com/github/gg2001/transformer-circuits/blob/master/arena/%5B1_3_1%5D_Superposition_%26_Sparse_Autoencoders_(exercises).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [1.3.1] Superposition & Sparse AutoEncoders (exercises)

Colab: [exercises](https://colab.research.google.com/drive/1DSqMihSkocF4WtLtazpIpZoUzrlu3Vdb?usp=sharing) | [solutions](https://colab.research.google.com/drive/1mHKZpkhYAr0WWAQo2Y6pXL08yNfJHOVx?usp=sharing)

ARENA 3.0 [Streamlit page](https://arena3-chapter1-transformer-interp.streamlit.app/[1.3.1]_Superposition_Sparse_Autoencoders)

Please send any problems / bugs on the `#errata` channel in the [Slack group](https://join.slack.com/t/arena-uk/shared_invite/zt-2noug8mpy-TRYbCnc3pzj7ITNrZIjKww), and ask any questions on the dedicated channels for this chapter of material.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/galaxies.jpeg" width="350">


# Introduction

Superposition is a crucially important concept for understanding how transformers work. A definition from Neel Nanda's glossary:

> Superposition is when a model represents more than n features in an $n$-dimensional activation space. That is, features still correspond to directions, but **the set of interpretable directions is larger than the number of dimensions**.

Why should we expect something like this to happen? In general, the world has way more features than the model has dimensions of freedom, and so we can't have a one-to-one mapping between features and values in our model. But the model has to represent these features somehow. Hence, it comes up with techniques for cramming multiple features into fewer dimensions (at the cost of adding noise and interference between features).

In these exercises, we recommend being liberal with the sections you skip (in particular, we recommend skipping sections 4 and 5 at least, unless you really want to dive deep into these topics). If you really want to speedrun SAEs (e.g. because you want to get to the exercises in 1.3.2) then at minimum we recommend section 1️⃣ and the first half of section 5️⃣ (up to but not including resampling).

Unlike many other topics in this chapter, there's quite a bit of theory which needs to be understood before we start making inferences from the results of our coding experiments. A key point to make here is that, perhaps more so than any other section in this chapter, we really don't understand superposition that well at all! It's hard to point to the seminal work in this field because we don't really know what the critical new insights will look like. That being said, we hope this material gives you enough directions to pursue when you're finished!

## Content & Learning Objectives


### 1️⃣, 2️⃣, 3️⃣ Toy Models of Superposition: basics

In these sections, you'll be exposed to Anthropic's toy models of superposition. You'll learn about the key ideas of superposition and why it presents a problem for interpreting neural networks, and you'll experiment with different ways of visualising superposition. You'll also see how properties like sparsity and importance of features affect the degree and nature of superposition.

> ##### Learning objectives
>
> - Understand the concept of superposition, and why models need to do it
> - Understand the difference between superposition and polysemanticity
> - Understand the difference between neuron and bottleneck superposition (or computational and representational superposition)
> - Build & train the toy model from Anthropic's paper, replicate the main results
> - See how superposition varies when you change the following characteristics of the features:
>   - Importance
>   - Sparsity
>   - Correlation

<br>

### 4️⃣ Feature Geometry

The section takes a slightly deeper dive into the geometry of superposition. It's not as essential as any exercises from the previous three sections, but should still be of interest and worth doing if you have time, or just want to dive more deeply into superposition.

> ##### Learning objectives
>
> - Understand the geometric intuitions behind superposition, and how they relate to the more general ideas of superposition in larger models

<br>

### 5️⃣ Sparse AutoEncoders in Toy Models

In this last section, you'll learn about sparse autoencoders, and how they might help us resolve problems of superposition. You'll train a sparse autoencoder on the toy model setup from earlier sections, and you'll also implement techniques like neuron resampling and different architectures (e.g. the Gated architecture from [DeepMind's paper](https://deepmind.google/research/publications/88147/)).

> ##### Learning objectives
>
> - Learn about sparse autoencoders, and how they might be used to disentangle features represented in superposition
> - Train your own SAEs on the toy models from earlier sections, and visualise the feature reconstruction process
> - Understand important SAE training strategies (e.g. resampling) and architecture variants (e.g. Gated, Jump ReLU)

<br>

### 6️⃣ Bonus

We end with a section of suggested bonus material & paper replications, like usual.

## Reading Material

* [200 COP in MI: Exploring Polysemanticity and Superposition](https://www.alignmentforum.org/posts/o6ptPu7arZrqRCxyz/200-cop-in-mi-exploring-polysemanticity-and-superposition), <b>15 mins</b>
    * Read the post, up to and including "Tips" (although some parts of it might make more sense after you've read the other things here).
* Neel Nanda's [Dynalist notes on superposition](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#z=3br1psLRIjQCOv2T4RN3V6F2), <b>10 mins</b>
    * These aren't long, you should skim through them, and also use them as a reference during these exercises.
* Anthropic's [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html), <b>20 mins</b>
    * You should read up to & including the "Summary: A Hierarchy of Feature Properties" section.
    * The first few sections ("Key Results", "Definitions and Motivation", and "Empirical Phenomena" are particularly important).
    * We'll also be going through other parts of this paper as we work through the exercises.

## Questions

Here are a set of questions (with some brief answers provided) which you should be able to answer for yourself after reading the above material. Seach for them on Neel's Dynalist notes if you didn't come across them during your reading.

What is a **privileged basis**? Why should we expect neuron activations to be privileged by default? Why *shouldn't* we expect the residual stream to be privileged?

<details>
<summary>Answer</summary>

A privileged basis is one where the **standard basis directions are meaningful** (due to the structure of computation being done on that basis). This doesn't necessarily mean that the basis is interpretable.

**Neurons**

Neuron activations are privileged because of the **elementwise nonlinear function that gets applied**. ReLU is easily described in the standard basis, e.g. in 2D:

$$
\begin{bmatrix} x \\ y \end{bmatrix}
\to
\begin{bmatrix} \max(x, 0) \\ \max(y, 0) \end{bmatrix}
$$

but if you redefine a basis $x' = (x+y)/\sqrt{2}$, $y' = (x-y)/\sqrt{2}$, then describing ReLU in this new basis becomes really messy. More importantly, we now get interference between the components $x'$ and $y'$, i.e. the ReLU is no longer acting on them independently.

$$
\begin{bmatrix} x' \\ y' \end{bmatrix}
\to
\frac{1}{\sqrt{2}} \begin{bmatrix} \max(x, 0) + \max(y, 0) \\ \max(x, 0) - \max(y, 0) \end{bmatrix}
=
\frac{1}{2} \begin{bmatrix} \max(x'+y', 0) + \max(x'-y', 0) \\ \max(x'+y', 0) - \max(x'-y', 0) \end{bmatrix}
$$

**Residual stream**

The residual stream is not privileged because anything that reads from it and writes to it uses a linear map. As a thought experiment, if we changed all the writing matrices (i.e. $W_{out}$ in the MLP layers and $W_O$ in the attention layers) to $W \to W R$, and all the reading matrices (i.e. $W_{in}$ in the MLP layers and $W_Q$, $W_K$, $W_V$ in the attention layers) to $W \to W R^{-1}$ where $R$ is some arbitrary rotation matrix, then the model's computation would be unchanged. Since the matrix $R$ is arbitrary, it can change the basis in any way it wants, so that basis can't be privileged.

To put this another way - if you claimed "I think the 47th element of the residual stream encoded some special information e.g. the plurality of the noun at that sequence position", I could call bullshit on your claim, because this thought experiment shows that any basis direction could just as easily be rotated & distributed as a linear combination of several different basis directions without fundamentally changing the computation done by the transformer. The same does not apply to neurons, because a rotation / change of basis would change the nature of computation done on them.

**Summary**

**Something is a privileged basis if it is not rotation-independent**, i.e. the nature of computation done on it means that the **basis directions have some special significance.**

Common misconception: privileged basis is equivalent to interpretable basis. This is **NOT true** (although it is the case that a basis must be privileged if the individual basis directions have some interpretable meaning; this is necessary but not sufficient).

</details>

What is the difference between **superposition** and **polysemanticity**?

<details>
<summary>Answer</summary>

Polysemanticity happens when one neuron corresponds to multiple features (see [here](https://distill.pub/2020/circuits/zoom-in/#:~:text=lot%20of%20effort.-,Polysemantic%20Neurons,-This%20essay%20may) for more discussion & examples). If we only had polysemanticity, this wouldn't really be a problem for us (there might exist a basis for features s.t. each basis vector corresponds to a single feature).

Superposition is when there are **more features than dimensions**. So it implies polysemanticity (because we must have dimensions representing more than one feature), but the converse is not true.

</details>


What are the **importance** and **sparsity** of features? Do you expect more or less polysemantic neurons if sparsity is larger?

<details>
<summary>Answer</summary>

**Importance** = how useful is this feature for achieving lower loss?

**Sparsity** = how frequently is it in the input data?

If sparsity is larger, then we expect more polysemantic neurons. This is because a single neuron can afford to represent several different sparse features (usually it'll only be representing one of them at any given time, so there won't be interference).
</details>

How would you define a **feature**?

<details>
<summary>Answer</summary>

There's no single correct answer to this. Many of the definitions are unfortunately circular (e.g. "a feature is a thing which could be represented by a neuron"). A few possible definitions are this one from Neel's [Dynalist notes](https://dynalist.io/d/n2ZWtnoYHrU1s4vnFSAQ519J#q=feature):

> A feature is a property of an input to the model, or some subset of that input (eg a token in the prompt given to a language model, or a patch of an image).

or this similar one from Chris Olah's [Distill circuits Thread](https://distill.pub/2020/circuits/zoom-in/):

> A feature is a a scalar function of the input. In this essay, neural network features are directions, and often simply individual neurons. We claim such features in neural networks are typically meaningful features which can be rigorously studied. A **meaningful feature** is one that genuinely responds to an articulable property of the input, such as the presence of a curve or a floppy ear.
</details>


## Setup (don't read, just run!)


In [ ]:
try:
    import google.colab # type: ignore
    IN_COLAB = True
except:
    IN_COLAB = False

import os, sys
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"

if IN_COLAB:
    # Install packages
    %pip install jaxtyping
    %pip install transformer_lens
    %pip install git+https://github.com/callummcdougall/eindex.git

    # Code to download the necessary files (e.g. solutions, test funcs)
    if not os.path.exists(f"/content/{chapter}"):
        !wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
        !unzip /content/main.zip 'ARENA_3.0-main/chapter1_transformer_interp/exercises/*'
        sys.path.append(f"/content/{repo}-main/{chapter}/exercises")
        os.remove("/content/main.zip")
        os.rename(f"{repo}-main/{chapter}", chapter)
        os.rmdir(f"{repo}-main")
        os.chdir(f"{chapter}/exercises")
else:
    raise Exception("If running from VSCode, you should copy code from the Streamlit page, not the Colab.")

In [ ]:
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Callable, Literal
import einops
import numpy as np
import torch as t
from jaxtyping import Float
from torch import Tensor, nn
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from tqdm.auto import tqdm
from IPython.display import display, HTML

# Make sure exercises are in the path
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = (exercises_dir / "part31_superposition_and_saes").resolve()
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

import part31_superposition_and_saes.utils as utils
import part31_superposition_and_saes.tests as tests
from plotly_utils import line, imshow

device = t.device("cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

# 1️⃣ TMS: Superposition in a Nonprivileged Basis

## Toy Model setup

In this section, we'll be examining & running experiments on the toy model studied in [Anthropic's paper](https://transformer-circuits.pub/2022/toy_model/index.html).

You can follow along with the paper from the [Demonstrating Superposition](https://transformer-circuits.pub/2022/toy_model/index.html#demonstrating) section onwards; it will approximately follow the order of the sections in this notebook.

This paper presented a very rudimentary model for **bottleneck superposition** - when you try and represent more than $n$ features in a vector space of dimension $n$. The model is as follows:

* We take a 5-dimensional input $x$
* We map it down into 2D space
* We map it back up into 5D space (using the transpose of the first matrix)
* We add a bias and ReLU

$$
\begin{aligned}
h &= W x \\
x' &= \operatorname{ReLU}(W^T h + b)
\end{aligned}
$$

### What's the motivation for this setup?

The input $x$ represents our five features (they're uniformly sampled between 0 and 1).

Each feature can have **importance** and **sparsity**. Recall our earlier definitions:

* **Importance** = how useful is this feature for achieving lower loss?
* **Sparsity** = how frequently is it in the input data?

This is realised in our toy model as follows:

* **Importance** = the coefficient on the weighted mean squared error between the input and output, which we use for training the model
    * In other words, our loss function is $L = \sum_x \sum_i I_i (x_i - x_i^\prime)^2$, where $I_i$ is the importance of feature $i$.
* **Sparsity** = the probability of the corresponding element in $x$ being zero
    * In other words, this affects the way our training data is generated (see the method `generate_batch` in the `Module` class below)
    * We often refer to **feature probability** (1 minus sparsity) rather than sparsity

The justification for using $W^T W$ is as follows: we can think of $W$ (which is a matrix of shape `(2, 5)`) as a grid of "overlap values" between the features and bottleneck dimensions. The values of the 5x5 matrix $W^T W$ are the dot products between the 2D representations of each pair of features. To make this intuition clearer, imagine each of the columns of $W$ were unit vectors, then $W^T W$ would be a matrix of cosine similarities between the features (with diagonal elements equal to 1, because the similarity of a feature with itself is 1). To see this for yourself:


In [ ]:
t.manual_seed(2)

W = t.randn(2, 5)
W_normed = W / W.norm(dim=0, keepdim=True)

imshow(W_normed.T @ W_normed, title="Cosine similarities of each pair of 2D feature embeddings", width=600)

To put it another way - if the columns of $W$ were orthogonal, then $W^T W$ would be the identity. This can't actually be the case because $W$ is a 2x5 matrix, but its columns can be "nearly orthgonal" in the sense of having pairwise cosine similarities close to 0.

<details>
<summary>

Question - can you prove that $W^T W$ can't be the identity when $W$ is not a square matrix?
</summary>

Proof #1: the rank of a matrix product $AB$ is upper-bounded by the maximum of the two factors $A$ and $B$. In the case of $W^T W$, both matrices have rank at most 2, so the product has rank at most 2.

Proof #2: for any vector $x$, $W^T W x = W^T (Wx)$ is in the span of the columns of $W^T$, which is vector space with rank 2.

</details>


Another nice thing about using two bottleneck dimensions is that we get to visualise our output! We've got a few helper functions for this purpose.


In [ ]:
utils.plot_features_in_2d(
    W_normed.unsqueeze(0), # shape [instances=1 d_hidden=2 features=5]
)

Compare this plot to the `imshow` plot above, and make sure you understand what's going on here (and how the two plots relate to each other). A lot of the subsequent exercises run with this idea of a geometric interpretation of the model's features and bottleneck dimensions.

<details>
<summary>Help - I'm confused about how these plots work.</summary>

As mentioned, you can view $W$ as being a set of five 2D vectors, one for each of our five features. The heatmap shows us the cosine similarities between each pair of these vectors, and the second plot shows us these five vectors in 2D space.

In the example above, we can see two pairs of vectors (the 1st & 2nd, and the 0th & 4th) have very high cosine similarity. This is reflected in the 2D plot, where these features are very close to each other (the 0th feature is the darkest color, the 4th feature is the lightest).

</details>

### Defining our model

Below is some code for your model (with most methods not filled out yet). It should be familiar to you if you've already built simple neural networks earlier in this course.

Some notes on the initialization method, which is filled out for you:

#### Weights & instances

The `Config` class has an `n_inst` class. This is so we can optimize multiple models at once in a single training loop (this'll be useful later on). You should treat this as basically like a batch dimension for your weights: each of your weights/biases will actually be `n_inst` separate weights/biases stacked along the zeroth dimension, and each of these will be trained independently, on different data, in parallel (using the same optimizer).

We initialize weights `W` and `b_final`, which correspond to $W$ and $b$ in the Anthropic paper.

#### Feature probability and sparsity

The `feature_probability` and `importance` arguments correspond to sparsity and importance of features.

We have the relation  `feature_probability = 1 - sparsity`. We'll usually refer to the feature probability rather than the sparsity, since this is easier. We'll often be dealing with very small values $p = 1 - S \approx 0$. The feature probability is used to generate our training data; the importance is used in our loss function (see later for both of these). The default is `feature_probability = None`, which means $p = 1$ (no sparsity).

The `importance` argument is used when calculating loss (see later exercise). The default is `importance = None` which results in uniform importance.

In the `__init__` method, we have code to broadcast `feature_probability` and `importance`, so that by the end they both always have shape `(n_inst, n_features)`.

### Exercise - implement `forward`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-20 minutes on this exercise.
```

For now, you just need to fill in the `forward` method. As the exercises go on, you'll fill in some more of these functions, but for now you can ignore the others.

In [ ]:
def linear_lr(step, steps):
    return (1 - (step / steps))

def constant_lr(*_):
    return 1.0

def cosine_decay_lr(step, steps):
    return np.cos(0.5 * np.pi * step / (steps - 1))


@dataclass
class Config:
    # We optimize n_inst models in a single training loop to let us sweep over sparsity or importance
    # curves efficiently. You should treat the number of instances `n_inst` like a batch dimension,
    # but one which is built into our training setup. Ignore the latter 3 arguments for now, they'll
    # return in later exercises.
    n_inst: int
    n_features: int = 5
    d_hidden: int = 2
    n_correlated_pairs: int = 0
    n_anticorrelated_pairs: int = 0
    feat_mag_distn: Literal["unif", "jump"] = "unif"


class Model(nn.Module):
    W: Float[Tensor, "inst d_hidden feats"]
    b_final: Float[Tensor, "inst feats"]

    # Our linear map (for a single instance) is x -> ReLU(W.T @ W @ x + b_final)

    def __init__(
        self,
        cfg: Config,
        feature_probability: float | Tensor = 0.01,
        importance: float | Tensor = 1.0,
        device=device,
    ):
        super(Model, self).__init__()
        self.cfg = cfg

        if isinstance(feature_probability, float):
            feature_probability = t.tensor(feature_probability)
        self.feature_probability = feature_probability.to(device).broadcast_to(
            (cfg.n_inst, cfg.n_features)
        )
        if isinstance(importance, float):
            importance = t.tensor(importance)
        self.importance = importance.to(device).broadcast_to((cfg.n_inst, cfg.n_features))

        self.W = nn.Parameter(
            nn.init.xavier_normal_(t.empty((cfg.n_inst, cfg.d_hidden, cfg.n_features)))
        )
        self.b_final = nn.Parameter(t.zeros((cfg.n_inst, cfg.n_features)))
        self.to(device)


    def forward(
        self,
        features: Float[Tensor, "... inst feats"],
    ) -> Float[Tensor, "... inst feats"]:
        # YOUR CODE HERE
        raise NotImplementedError()


    def generate_batch(self, batch_size) -> Float[Tensor, "batch inst feats"]:
        """
        Generates a batch of data.
        """
        # You'll fill this in later
        raise NotImplementedError()


    def calculate_loss(
        self,
        out: Float[Tensor, "batch inst feats"],
        batch: Float[Tensor, "batch inst feats"],
    ) -> Float[Tensor, ""]:
        """
        Calculates the loss for a given batch (as a scalar tensor), using this loss described in the
        Toy Models of Superposition paper:

            https://transformer-circuits.pub/2022/toy_model/index.html#demonstrating-setup-loss

        Note, `self.importance` is guaranteed to broadcast with the shape of `out` and `batch`.
        """
        # You'll fill this in later
        raise NotImplementedError()


    def optimize(
        self,
        batch_size: int = 1024,
        steps: int = 10_000,
        log_freq: int = 50,
        lr: float = 1e-3,
        lr_scale: Callable[[int, int], float] = constant_lr,
    ):
        """
        Optimizes the model using the given hyperparameters.
        """
        optimizer = t.optim.Adam(list(self.parameters()), lr=lr)

        progress_bar = tqdm(range(steps))

        for step in progress_bar:
            # Update learning rate
            step_lr = lr * lr_scale(step, steps)
            for group in optimizer.param_groups:
                group["lr"] = step_lr

            # Optimize
            optimizer.zero_grad()
            batch = self.generate_batch(batch_size)
            out = self(batch)
            loss = self.calculate_loss(out, batch)
            loss.backward()
            optimizer.step()

            # Display progress bar
            if step % log_freq == 0 or (step + 1 == steps):
                progress_bar.set_postfix(loss=loss.item() / self.cfg.n_inst, lr=step_lr)


tests.test_model(Model)

### Exercise - implement `generate_batch`

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

Next, you should implement the function `generate_batch` above. This should return a tensor of shape `(n_batch, instances, features)`, where:

* The `instances` and `features` values are taken from the model config,
* Each feature is present with probability `self.feature_probability`,
* For each present feature, its **magnitude** is sampled from a uniform distribution between 0 and 1.

Make sure you understand this function well (we recommend looking at the solutions even after you pass the tests), because we'll be making more complicated versions of this function in the section on correlations.

Remember, you can assume `model.feature_probability` has shape `(n_inst, n_features)`.

When you've implemented this function, run the code below to test it.

In [ ]:
tests.test_generate_batch(Model)

## Training our model


The details of training aren't very conceptually important, so we've given you most of the code to train the model below. We use **learning rate schedulers** to control the learning rate as the model trains - you'll use this later on during the RL chapter.

### Exercise - implement `calculate_loss`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-10 minutes on this exercise.
```

You should fill in the `calculate_loss` function below. The loss function **for a single instance** is given by:

$$
L=\frac{1}{BF}\sum_x \sum_i I_i\left(x_i-x_i^{\prime}\right)^2
$$

where:

* $B$ is the batch size,
* $F$ is the number of features,
* $x_i$ are the inputs and $x_i'$ are the model's outputs,
* $I_i$ is the importance of feature $i$,
* $\sum_i$ is a sum over features,
* $\sum_x$ is a sum over the elements in the batch.

For the general case, we sum this formula over all instances.

<details>
<summary>Question - why do you think we take the mean over the feature and batch dimensions, but we sum over the instances dimension?</summary>

We take the mean over batch size because this is standard for loss functions (and means we don't have to use a different learning rate for different batch sizes).

We take the mean over the feature dimension because that's [normal for MSE loss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html).

We sum over the instances dimension because we want to train each instance independently, and at the same rate as we would train a single instance.

</details>

In [ ]:
tests.test_calculate_loss(Model)

Now, we'll reproduce a version of the figure from the introduction, although with a slightly different version of the code.

A few notes:

* The `importance` argument is the same for all instances. It takes values between 1 and ~0.66 for each feature (so for every instance, there will be some features which are more important than others).
* The `feature_probability` is the same for all features, but it varies across instances. In other words, we're runnning several different experiments at once, and we can compare the effect of having larger feature sparsity in these experiments.


In [ ]:
cfg = Config(n_inst=8, n_features=5, d_hidden=2)

# importance varies within features for each instance
importance = (0.9 ** t.arange(cfg.n_features))

# sparsity is the same for all features in a given instance, but varies over instances
feature_probability = (50 ** -t.linspace(0, 1, cfg.n_inst))

line(importance, width=600, height=400, title="Importance of each feature (same over all instances)", labels={"y": "Feature importance", "x": "Feature"})
line(feature_probability, width=600, height=400, title="Feature probability (varied over instances)", labels={"y": "Probability", "x": "Instance"})

In [ ]:
model = Model(
    cfg=cfg,
    device=device,
    importance=importance[None, :],
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

In [ ]:
utils.plot_features_in_2d(
    model.W,
    colors=model.importance,
    title=f"Superposition: {cfg.n_features} features represented in 2D space",
    subplot_titles=[f"1 - S = {i:.3f}" for i in feature_probability.squeeze()],
)

<details>
<summary>Click this dropdown to see what you should be getting from this visualisation.</summary>

<br>
<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/sp1.png" width="1400">

</details>

### Exercise - interpret these diagrams

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 10-20 minutes on this exercise.
```

Remember that for all these diagrams, the darker colors have lower importance and the lighter colors have higher importance. Also, the sparsity of all features is increasing as we move from left to right (at the far left there is no sparsity, at the far right feature probability is 5% for all features, i.e. sparsity of 95%).

<details>
<summary>Hint</summary>

For low sparsity, think about what the model would learn to do if all 5 features were present all the time. What's the best our model could do in this case, and how does that relate to the **importance** values?

For high sparsity, think about what the model would learn to do if there was always exactly one feature present. Does this make interference between features less of a problem?
</details>

<details>
<summary>Answer (intuitive)</summary>

When there is no sparsity, the model can never represent more than 2 features faithfully, so it makes sense for it to only represent the two most important features. It stores them orthogonally in 2D space, and sets the other 3 features to zero. This way, it can reconstruct these two features perfectly, and ignores all the rest.

When there is high sparsity, we get a pentagon structure. Most of the time at most one of these five features will be active, which helps avoid **interference** between features. When we try to recover our initial features by projecting our point in 2D space onto these five directions, most of the time when feature $i$ is present, we can be confident that our projection onto the $i$-th feature direction only captures this feature, rather than being affected by the presence of other features. We omit the mathematical details here.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/download (7).png" width="900">

The key idea here is that two forces are competing in our model: **feature benefit** (representing more thing is good!), and **interference** (representing things non-orthogonally is bad). The higher the sparsity, the more we can reduce the negative impact of interference, and so the trade-off skews towards "represent more features, non-orthogonally".

</details>


We can also generate a batch and visualise its embedding. Most interestingly, you should see that in the plots with high sparsity (to the right), we very rarely have interference between the five features, because most often $\leq 1$ of those features is present, and the model can recover it by projecting along the corresponding feature dimension without losing any information.

In [ ]:
with t.inference_mode():
    batch = model.generate_batch(200)
    hidden = einops.einsum(batch, model.W, "batch_size instances features, instances hidden features -> instances hidden batch_size")

utils.plot_features_in_2d(hidden, title = "Hidden state representation of a random batch of data")

<details>
<summary>Click this dropdown to see what you should be getting from this visualisation.</summary>

<br>
<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/hs1.png" width="1400">

</details>

## Visualizing features across varying sparsity


Now that we've got our pentagon plots and started to get geometric intuition for what's going on, let's scale things up! We're now operating in dimensions too large to visualise, but hopefully our intuitions will carry over.


In [ ]:
cfg = Config(n_inst=10, n_features=100, d_hidden=20)

importance = 100 ** -t.linspace(0, 1, cfg.n_features)
feature_probability = 20 ** -t.linspace(0, 1, cfg.n_inst)

line(importance, width=600, height=400, title="Importance of each feature (same over all instances)", labels={"y": "Feature importance", "x": "Feature"})
line(feature_probability, width=600, height=400, title="Feature probability (varied over instances)", labels={"y": "Probability", "x": "Instance"})

In [ ]:
model = Model(
    cfg=cfg,
    device=device,
    importance=importance[None, :],
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

Because we can't plot features in 2D anymore, we're going to use a different kind of visualisation:

* The **bottom row plots** shows a bar graph of all the features and their corresponding embedding norms $||W_i||$.
    * As we increase sparsity, the model is able to represent more features (i.e. we have more features with embedding norms close to 1).
    * We also color the bars according to whether they're orthogonal to other features (purple) or not (yellow). So we can see that for low sparsity most features are represented orthogonally (like our left-most plots above) but as we increase sparsity we transition to all features being represented non-orthogonally (like our right-most pentagon plots above).
* The **top row plots** show us the dot products between all pairs of feature vectors (kinda like the heatmaps we plotted at the start of this section).
    * This is another way of visualising the increasing interference between features as we increase sparsity.
    * Note that all these right hand plots represent **matrices with rank at most `d_hidden=20`**. The first few are approximately submatrices of the identity (because we perfectly reconstruct 20 features and delete the rest), but the later plots start to display inference as we plot more than 20 values (the diagonals of these matrices have more than 20 non-zero elements).

See the section [Basic Results](https://transformer-circuits.pub/2022/toy_model/index.html#demonstrating-basic-results) for more of an explanation of this graph and what you should interpret from it.

In [ ]:
utils.plot_features_in_Nd(
    model.W,
    height=800,
    width=1600,
    title="ReLU output model: n_features = 80, d_hidden = 20, I<sub>i</sub> = 0.9<sup>i</sup>",
    subplot_titles=[f"Feature prob = {i:.3f}" for i in feature_probability],
)

## Bonus - varying feature probability across features

In this section, we've only discussed varying feature probability across instances, and so most of the learned solutions have had uniformity (e.g. a uniform pentagon, or a uniform digon with one feature collapsed). But there's also a large set of non-uniform patterns which can be learned by our models. See the [corresponding section](https://transformer-circuits.pub/2022/toy_model/index.html#geometry-non-uniform) of the Anthropic paper, where they discuss in more detail what happens when a feature's importance is perturbed. Can you reproduce this result? Can you think of a setup which would result in a learned solution where all 5 features are represented, but two features are represented with very high cosine similarity? (You might want to return to this question at the end of the next section!).

In [ ]:
# YOUR CODE HERE - replicate Anthropic's peturbing results (optional)

# See the solutions Colab or Streamlit page for replication

# 2️⃣ TMS: Correlated / Anticorrelated Features

## Superposition with correlation

One major thing we haven't considered in our experiments is **correlation**. We could guess that superposition is even more common when features are **anticorrelated** (for a similar reason as why it's more common when features are sparse). Most real-world features are anticorrelated (e.g. the feature "this is a sorted Python list" and "this is some text in an edgy teen vampire romance novel" are probably anticorrelated - that is, unless you've been reading some pretty weird fanfics).

In this section, you'll define a new data-generating function for correlated features, and run the same experiments as in the first section.


### Exercise - implement `generate_correlated_batch`

```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵⚪⚪⚪

You should spend up to 20-40 minutes on this exercise.

The exercise itself is not conceptually important, and it is a bit fiddly / delicate, so you should definitely look at the
solutions if you get stuck. Understanding the results and why they occur is more important than the implementation!
```

You should now fill in the three methods `generate_correlated_features`, `generate_anticorrelated_features` and `generate_uncorrelated_features` in the `Model` class, which are created to generate correlated / anticorrelated data. We've given you a new `generate_batch` function which returns the aggregation from all of these methods.

Note, in the correlated & anticorrelated cases you can assume that the feature probability is the same for all features in each instance. We start these functions by asserting this for you, and creating a vector `p` which contains this feature probability for each instance (which is what you should use instead of `model.feature_probability`). The same is also true for the uncorrelated case, when the number of uncorrelated features we're generating is less than `cfg.n_features` (since if not, it's fine to use the full `self.feature_probability` tensor).

You'll also need to be careful with your probabilities in the anticorrelated case. For example, if you do the following for your pair of features 1 & 2:

```python
feat1_is_present = t.rand() < p
feat2_is_present = t.rand() < p & ~feat1_is_present
```

then your `feat2` probability will actually be `p * (1 - p)` rather than the intended `p`. You want to try and make both features have probability `p`, while _also_ ensuring that they are never both active at the same time! The hints provide some guidance on how you can implement this (it's a bit fiddly and not very conceptually important!).

For more details, you can read the [experimental details in Anthropic's paper](https://transformer-circuits.pub/2022/toy_model/index.html#geometry-correlated-setup), where they describe how they setup correlated and anticorrelated sets.

<details>
<summary>Help - I'm confused about how to implement the correlated features function.</summary>

Try first creating a boolean mask of shape `(batch_size, n_inst, n_correlated_pairs)` representing whether the pair is present, then repeating that mask across feature pairs with `einops.repeat`.

</details>

<details>
<summary>Help - I'm confused about how to implement the anticorrelated features function.</summary>

Here are 2 suggested methods:

1. Create a boolean mask of shape `(batch_size, n_inst, n_anticorrelated_pairs)` with probability $2p$, which represents whether *either* feature is present - and where true, we choose the present feature uniform randomly from the pair. This works because both features will have probability $2p \times 0.5 = p$.
2. Create 2 boolean masks `M1, M2` both of shape `(batch_size, n_inst, n_anticorrelated_pairs)` with probability $p$ and $p / (1 - p)$ respectively. Set the first feature to be present where `M1` is true, and the second feature to be present where `~M1 && M2` is true. This works because the first feature will have probability $p$, and the second will have probability $\frac{(1 - p)p}{(1 - p)} = p$.

The solutions use a method like (2), but either is valid.

</details>

In [ ]:
def generate_correlated_features(
    self, batch_size: int, n_correlated_pairs: int
) -> Float[Tensor, "batch inst 2*n_correlated_pairs"]:
    """
    Generates a batch of correlated features. For each pair `batch[i, j, [2k, 2k+1]]`, one of
    them is non-zero if and only if the other is non-zero.
    """
    assert t.all((self.feature_probability == self.feature_probability[:, [0]]))
    p = self.feature_probability[:, [0]]  # shape (n_inst, 1)

    # YOUR CODE HERE!
    raise NotImplementedError()


def generate_anticorrelated_features(
    self, batch_size: int, n_anticorrelated_pairs: int
) -> Float[Tensor, "batch inst 2*n_anticorrelated_pairs"]:
    """
    Generates a batch of anti-correlated features. For each pair `batch[i, j, [2k, 2k+1]]`, each
    of them can only be non-zero if the other one is zero.
    """
    assert t.all((self.feature_probability == self.feature_probability[:, [0]]))
    p = self.feature_probability[:, [0]]  # shape (n_inst, 1)

    assert p.max().item() <= 0.5, "For anticorrelated features, must have 2p < 1"

    # YOUR CODE HERE!
    raise NotImplementedError()


def generate_uncorrelated_features(self, batch_size: int, n_uncorrelated: int) -> Tensor:
    """
    Generates a batch of uncorrelated features.
    """
    if n_uncorrelated == self.cfg.n_features:
        p = self.feature_probability
    else:
        assert t.all((self.feature_probability == self.feature_probability[:, [0]]))
        p = self.feature_probability[:, [0]]  # shape (n_inst, 1)

    # YOUR CODE HERE!
    raise NotImplementedError()


def generate_batch(self, batch_size) -> Float[Tensor, "batch inst feats"]:
    """
    Generates a batch of data, with optional correlated & anticorrelated features.
    """
    n_corr_pairs = self.cfg.n_correlated_pairs
    n_anti_pairs = self.cfg.n_anticorrelated_pairs
    n_uncorr = self.cfg.n_features - 2 * n_corr_pairs - 2 * n_anti_pairs

    data = []
    if n_corr_pairs > 0:
        data.append(self.generate_correlated_features(batch_size, n_corr_pairs))
    if n_anti_pairs > 0:
        data.append(self.generate_anticorrelated_features(batch_size, n_anti_pairs))
    if n_uncorr > 0:
        data.append(self.generate_uncorrelated_features(batch_size, n_uncorr))
    batch = t.cat(data, dim=-1)
    return batch


Model.generate_correlated_features = generate_correlated_features
Model.generate_anticorrelated_features = generate_anticorrelated_features
Model.generate_uncorrelated_features = generate_uncorrelated_features
Model.generate_batch = generate_batch

The code below tests your function, by generating a large number of batches and measuring them statistically.


In [ ]:
cfg = Config(n_inst=30, n_features=4, d_hidden=2, n_correlated_pairs=1, n_anticorrelated_pairs=1)

feature_probability = 10 ** -t.linspace(0.5, 1, cfg.n_inst).to(device)

model = Model(cfg=cfg, device=device, feature_probability=feature_probability[:, None])

# Generate a batch of 4 features: first 2 are correlated, second 2 are anticorrelated
batch = model.generate_batch(batch_size=100_000)
corr0, corr1, anticorr0, anticorr1 = batch.unbind(dim=-1)

assert ((corr0 != 0) == (corr1 != 0)).all(), "Correlated features should be active together"
assert (
    ((corr0 != 0).float().mean(0) - feature_probability).abs().mean() < 0.002
), "Each correlated feature should be active with probability `feature_probability`"

assert (
    (anticorr0 != 0) & (anticorr1 != 0)
).int().sum().item() == 0, "Anticorrelated features should never be active together"
assert (
    ((anticorr0 != 0).float().mean(0) - feature_probability).abs().mean() < 0.002
), "Each anticorrelated feature should be active with probability `feature_probability`"

We can also visualise these features, in the form of a bar chart. You should see the correlated features always co-occurring, and the anticorrelated features never co-occurring.

<details>
<summary>What you should see when you run the code below</summary>


<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/bar-cooccur.png" width="800">

</details>

In [ ]:
# Generate a batch of 4 features: first 2 are correlated, second 2 are anticorrelated
batch = model.generate_batch(batch_size=1)
correlated_feature_batch, anticorrelated_feature_batch = batch.split(2, dim=-1)

# Plot correlated features
utils.plot_correlated_features(correlated_feature_batch, title="Correlated feature pairs: should always co-occur")
utils.plot_correlated_features(anticorrelated_feature_batch, title="Anti-correlated feature pairs: should never co-occur")

Now, let's try training our model & visualising features in 2D, when we have 2 pairs of correlated features (matching the [first row of the correlation figure](https://transformer-circuits.pub/2022/toy_model/index.html#geometry-organization) in the Anthropic paper).

In [ ]:
cfg = Config(n_inst=5, n_features=4, d_hidden=2, n_correlated_pairs=2)

# All same importance, very low feature probabilities (ranging from 5% down to 0.25%)
importance = t.ones(cfg.n_features, dtype=t.float, device=device)
feature_probability = 400 ** -t.linspace(0.5, 1, cfg.n_inst)

model = Model(
    cfg=cfg,
    device=device,
    importance=importance[None, :],
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

In [ ]:
utils.plot_features_in_2d(
    model.W,
    colors=["blue"] * 2 + ["limegreen"] * 2,
    title="Correlated feature sets are represented in local orthogonal bases",
    subplot_titles=[f"1 - S = {i:.3f}" for i in feature_probability],
)

### Exercise - generate more correlated feature plots

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to ~10 minutes on this exercise.

It should just involve changing the parameters in your code above.
```

You should now reproduce the second and third rows from the paper's [correlation figure](https://transformer-circuits.pub/2022/toy_model/index.html#geometry-organization). You may not get exactly the same results as the paper, but they should still roughly match (e.g. you should see no antipodal pairs in the code above, but you should see at least some when you test the anticorrelated sets, even if not all of them are antipodal). You can look at the solutions colab to see some examples.

<details>
<summary>Question - for the anticorrelated feature plots, you'l have to increase the feature probability to something like ~10%, or else you won't always form antipodal pairs. Why do you think this is?</summary>

If sparsity is small / feature prob is large, then interference between the two pairs of anticorrelated features is a problem. If two features from different pairs are in the same subspace (because they're antipodal) the model is more likely to keep looking for a better solution.

On the other hand, if sparsity is very large / feature probability is close to zero, then the negative effect of interference is much smaller. So the difference in loss between the solutions where the antipodal pairs are / aren't the same as the anticorrelated pairs is much smaller, and the model is more likely to just settle on whichever solution it finds first.

</details>

In [ ]:
# YOUR CODE HERE - generate more correlated feature plots

See the [Colab notebook](https://colab.research.google.com/drive/1mHKZpkhYAr0WWAQo2Y6pXL08yNfJHOVx?usp=sharing) for some sample code & outputs for these plots.

# 3️⃣ TMS: Superposition in a Privileged Basis


## Introduction

So far, we've explored superposition in a model without a privileged basis. We can rotate the hidden activations arbitrarily and, as long as we rotate all the weights, have the exact same model behavior. That is, for any ReLU output model with weights
$W$, we could take an arbitrary orthogonal matrix $O$ and consider the model $W' = OW$. Since $(OW)^T(OW) = W^T W$, the result would be an identical model!

Models without a privileged basis are elegant, and can be an interesting analogue for certain neural network representations which don't have a privileged basis – word embeddings, or the transformer residual stream. But we'd also (and perhaps primarily) like to understand neural network representations where there are neurons which do impose a privileged basis, such as transformer MLP layers or conv net neurons.

Our goal in this section is to explore the simplest toy model which gives us a privileged basis. There are at least two ways we could do this: we could add an activation function or apply $L_1$ regularization to the hidden layer. We'll focus on adding an activation function, since the representation we are most interested in understanding is hidden layers with neurons, such as the transformer MLP layer.

This gives us the following "ReLU hidden layer" model. It's the simplest one we can use which is still likely to give us a privileged basis; we just take our previous setup and apply ReLU to the hidden layer.

$$
\begin{aligned}
h & =\operatorname{ReLU}(W x) \\
x^{\prime} & =\operatorname{ReLU}\left(W^T h+b\right)
\end{aligned}
$$

### Exercise - implement `NeuronModel`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to ~10 minutes on this exercise.
```

In this section, you'll replicate the [first set of results](https://transformer-circuits.pub/2022/toy_model/index.html#demonstrating-setup-loss:~:text=model%20and%20a-,ReLU%20hidden%20layer%20model,-%3A) in the Anthropic paper on studying superposition in a privileged basis. To do this, you'll need a new `NeuronModel` class. It can inherit most methods from the `Model` class, but you'll need to redefine the `forward` method to include an intermediate ReLU.

In [ ]:
class NeuronModel(Model):
    def forward(
        self,
        features: Float[Tensor, "... instances features"]
    ) -> Float[Tensor, "... instances features"]:
        # YOUR CODE HERE
        raise NotImplementedError()


tests.test_neuron_model(NeuronModel)


tests.test_neuron_model(NeuronModel)

Once you've passed these tests, you can run the cells below to train the model in the same way as before. We use just one instance, with zero sparsity and uniform importance.

We also visualize the matrix $W$. In these plots, we make it so the top-row visualisation is of $W$ rather than $W^T W$ - we can get away with this now because (unlike before) the individual elements of $W$ *are* meaningful. We're working with a **privileged basis**, and $W$ connects features to basis-aligned neurons.

In [ ]:
cfg = Config(n_inst=7, n_features=10, d_hidden=5)

importance = 0.75 ** t.arange(1, 1 + cfg.n_features)
feature_probability = t.tensor([0.75, 0.35, 0.15, 0.1, 0.06, 0.02, 0.01])

model = NeuronModel(
    cfg=cfg,
    device=device,
    importance=importance[None, :],
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

In [ ]:
utils.plot_features_in_Nd(
    model.W,
    height=600,
    width=1000,
    title=f"Neuron model: {cfg.n_features=}, {cfg.d_hidden=}, I<sub>i</sub> = 0.75<sup>i</sup>",
    subplot_titles=[f"1 - S = {i:.2f}" for i in feature_probability.squeeze()],
    neuron_plot=True,
)

### Exercise - interpret these plots

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 10-15 minutes on this exercise.
```

The first row shows plots of $W$. The rows are features, the columns are hidden dimensions (neurons).

The second row shows stacked weight plots: in other words, each column is a neuron, and the values in a column are the exposures of the features to that particular neuron. In these plots, each feature is colored differently based on its interference with other features (dark blue means the feature is orthogonal to all other features, and lighter colors means the sum of squared dot products with other features is large).

What is your interpretation of these plots? You should discuss things like monosemanticity / polysemanticity and how this changes with increasing sparsity.

<details>
<summary>Explanation for some of these plots</summary>

**Low sparsity / high feature probability**

With very low sparsity (feature prob $\approx 1$), we get no superposition: every feature is represented faithfully by a different one of the model's neurons, or not represented at all. In other words, we have **pure monosemanticity**.

In the heatmaps, we see a diagonal plot (up to rearrangement of neurons), i.e. each of the 5 most important features has a corresponding neuron which detects that particular feature, and no other.

In the bar charts, we see this monosemanticity represented: each neuron has just one feature exposed to it.

**Medium sparsity / medium feature probability**

At intermediate values, we get some monosemantic neurons, and some polysemantic ones. You should see reoccurring block patterns like these (up to rearrangements of rows and/or columns):

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/three_two2.png" width="130">

Can you see what geometric arrangements these correspond to? The answer is in the nested dropdown below.

<details>
<summary>Answer</summary>

The 3x2 block shows 3 features embedded in 2D space. Denoting the 3 features $i, j, k$ respectively, we can see that $j$ is represented along the direction $(1, 1)$ (orthogonal to the other two), and $i, k$ are represented as $(-1, 1)$ and $(1, -1)$ respectively (antipodal pairs).

As for the 3x3 block, it's actually 3 of the 4 points from a regular tetrahedron! This hints at an important fact which we'll explore in the next (optional) set of exercises: **superposition results in features organizing themselves into geometric structures**, which often represent uniform polyhedra.

</details>

The bar chart shows some neurons are starting to become polysemantic, with exposures to more than one feature.

**High sparsity / low feature probability**

With high sparsity, all neurons are polysemantic, and most / all features are represented in some capacity. The neurons aren't orthogonal (since we have way more features than neurons), but they don't need to be orthogonal: we saw in earlier sections how high sparsity can allow us to represent more features than we had dimensions. The same is true in this case.

Note - Anthropic [finds](https://transformer-circuits.pub/2022/toy_model/index.html#privileged-basis:~:text=The%20solutions%20are%20visualized%20below) that with very high sparsity, each feature will correspond to a pair of neurons. However, you may not find this for your own plots (I didn't!). This is because - as Anthropic mention - they trained many separate instances and took the ones with smallest loss, since these models proved more difficult to optimize than others in their toy model setup.

Overall, it looks a great deal like there are **neuron-level phase changes from monosemantic to polysemantic** as we increase the sparsity, mirroring the feature phase changes we saw earlier.

</details>

Try playing around with different settings (sparsity, importance). What kind of results do you get?

### Exercise (optional) - replicate plots more faithfully

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵⚪⚪⚪⚪

You should spend up to 10-25 minutes on this exercise, if you choose to do it.
```

Anthropic mention in their paper that they trained 1000 instances and chose the ones which achieved lowest loss. This is why your results might have differed from theirs, especially when the sparsity is very high / feature probability is very low.

Can you implement this "choose lowest loss" method in your own class? Some suggestions:

* The most basic way would be to modify the `optimize` function to return the loss per instance, and also use a for loop to run several `optimize` calls & at the end give you the best instances for each different level of sparsity.
* A much better way would be to train more instances at once (e.g. `N` instances per level of sparsity), then for each level of sparsity you can argmax over `N` at the end to get a single instance. This will be much faster (although you'll have to be careful not to train 1000 instances at once; your GPU might not support it!).
* To get very fancy, you could even add another dimension to the weight matrices, corresponding to this `N` dimension you argmax over. Then this "taking lowest-loss instance" behavior will be automatic.

## Computation in superposition

The example above was interesting, but in some ways it was also limited. The key problem here is that **the model doesn't benefit from the ReLU hidden layer**. Adding a ReLU does encourage the model to have a privileged basis, but since the model is trying to reconstruct the input (i.e. the identity, which is a linear function) it doesn't actually need to use the ReLU, and it will try anything it can to circumvent it - including learning biases which shift all the neurons into a positive regime where they behave linearly. This is a mark against using this toy model to study superposition.

To extend this point: we don't want to study boring linear functions like the identity, we want to study **how models perform (nonlinear) computation in superposition**. The MLP layer in a transformer isn't just a way to represent information faithfully and recover it; it's a way to perform computation on that information. So for this next section, we'll train a model to perform some non-linear computation. Specifically, we'll train our model to **compute the absolute value of inputs $x$**.

Our data $x$ are now sampled from the range $[-1, 1]$ rather than $[0, 1]$ (otherwise calculating the absolute value would be equivalent to reconstructing the input). This is about as simple as a nonlinear function can get, since $abs(x)$ is equivalent to $\operatorname{ReLU}(x) + \operatorname{ReLU}(-x)$. But since it's nonlinear, we can be sure that the model has to use the hidden layer ReLU.

### Exercise - implement `NeuronComputationModel`

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 20-30 minutes on this exercise.
```

You should fill in the `NeuronComputationModel` class below. Specifically, you'll need to fill in the `forward`, `generate_batch` and `calculate_loss` methods. Some guidance:

* The model's **forward function** is different - it has a ReLU hidden layer in its forward function (as described above & in the paper).
* The model's **data** is different - see the discussion above. Your `generate_batch` function should be rewritten - it will be the same as the first version of this function you wrote (i.e. without correlations) except for one difference: the value is sampled uniformly from the range $[-1, 1]$ rather than $[0, 1]$.
* The model's **loss function** is different. Rather than computing the importance-weighted $L_2$ error between the input $x$ and output $x'$, we're computing the importance-weighted $L_2$ error between $\operatorname{abs}(x)$ and $x'$. This should just require changing one line. The `optimize` function can stay the same, but it will now be optimizing this new loss function.

In [ ]:
class NeuronComputationModel(Model):
    W1: Float[Tensor, "inst d_hidden feats"]
    W2: Float[Tensor, "inst feats d_hidden"]
    b_final: Float[Tensor, "inst feats"]

    def __init__(
        self,
        cfg: Config,
        feature_probability: float | Tensor = 1.0,
        importance: float | Tensor = 1.0,
        device=device,
    ):
        super(Model, self).__init__()
        self.cfg = cfg

        if isinstance(feature_probability, float):
            feature_probability = t.tensor(feature_probability)
        self.feature_probability = feature_probability.to(device).broadcast_to(
            (cfg.n_inst, cfg.n_features)
        )
        if isinstance(importance, float):
            importance = t.tensor(importance)
        self.importance = importance.to(device).broadcast_to((cfg.n_inst, cfg.n_features))

        self.W1 = nn.Parameter(nn.init.kaiming_uniform_(t.empty((cfg.n_inst, cfg.d_hidden, cfg.n_features))))
        self.W2 = nn.Parameter(nn.init.kaiming_uniform_(t.empty((cfg.n_inst, cfg.n_features, cfg.d_hidden))))
        self.b_final = nn.Parameter(t.zeros((cfg.n_inst, cfg.n_features)))
        self.to(device)


    def forward(self, features: Float[Tensor, "... inst feats"]) -> Float[Tensor, "... inst feats"]:
        # YOUR CODE HERE
        raise NotImplementedError()


    def generate_batch(self, batch_size) -> Tensor:
        # YOUR CODE HERE
        raise NotImplementedError()


    def calculate_loss(
        self,
        out: Float[Tensor, "batch instances features"],
        batch: Float[Tensor, "batch instances features"],
    ) -> Float[Tensor, ""]:
        # YOUR CODE HERE
        raise NotImplementedError()


tests.test_neuron_computation_model(NeuronComputationModel)

Once you've passed these tests, you can run the code below to make the same visualisation as above.

You should see similar patterns: with very low sparsity most/all neurons are monosemantic, but more polysemantic neurons appear as sparsity increases (until all neurons are polysemantic). Another interesting observation: in the monosemantic (or mostly monosemantic) cases, for any given feature there will be some neurons which have positive exposures to that feature and others with negative exposure. This is because some neurons are representing the value $\operatorname{ReLU}(x_i)$ and others are representing the value of $\operatorname{ReLU}(-x_i)$ (as discussed above, we require both of these to compute the absolute value).

In [ ]:
cfg = Config(n_inst=7, n_features=100, d_hidden=40)

importance = 0.8 ** t.arange(1, 1 + cfg.n_features)
feature_probability = t.tensor([1.0, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001])

model = NeuronComputationModel(
    cfg=cfg,
    device=device,
    importance=importance[None, :],
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

In [ ]:
plot_features_in_Nd(
    model.W1,
    height = 800,
    width = 1600,
    title = f"Neuron computation model: n_features = {n_features}, d_hidden = {d_hidden}, I<sub>i</sub> = 0.75<sup>i</sup>",
    subplot_titles = [f"1 - S = {i:.3f}" for i in feature_probability.squeeze()],
    neuron_plot = True,
)

To further confirm that this is happening, we can color the values in the bar chart discretely by feature, rather than continuously by the polysemanticity of that feature. We'll use a feature probability of 50% for this visualisation, which is high enough to make sure each neuron is monosemantic. You should find that the input weights $W_1$ form pairs of antipodal neurons (i.e. ones with positive / negative exposures to that feature direction), but both of these neurons have positive output weights $W_2$ for that feature.

In [ ]:
cfg = Config(n_inst=7, n_features=100, d_hidden=40)

importance = 0.8 ** t.arange(1, 1 + cfg.n_features)
feature_probability = t.tensor([1.0, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001])

model = NeuronComputationModel(
    cfg=cfg,
    device=device,
    importance=importance[None, :],
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

In [ ]:
utils.plot_features_in_Nd(
    model.W1,
    height=800,
    width=1400,
    title=f"Neuron computation model: {cfg.n_features=}, {cfg.d_hidden=}, I<sub>i</sub> = 0.75<sup>i</sup>",
    subplot_titles=[f"1 - S = {i:.3f}<br>" for i in feature_probability.squeeze()],
    neuron_plot=True,
)

## Bonus - the asymmetric superposition motif

In the [corresponding section](https://transformer-circuits.pub/2022/toy_model/index.html#computation-asymmetric-motif) of Anthropic's paper, they discuss a particular quirk of this toy model in detail. Their section explains it in more detail than we will here (including some visual explanations), but we'll provide a relatively brief explanation here.

> When we increase sparsity in our model & start to get superposed features, we don't always have monosemantic neurons which each calculate either $\operatorname{ReLU}(x_i)$ or $\operatorname{ReLU}(-x_i)$ for some feature $i$. Instead, we sometimes have **asymmetric superposition, where a single neuron detects two different features $i$ and $j$, and stores these features with different magnitudes (assume the $W_1$ vector for feature $i$ is much larger). The $W_2$ vectors have flipped magnitudes (i.e. the vector for $j$ is much larger). When $i$ is present and $j$ is not, there's no problem, because the output for feature $i$ is `large * small` (correct size) and for $j$ is `small * small` (near zero). But when $j$ is present and $i$ is not, the output for feature $j$ is `small * large` (correct size) and for $i$ is `large * large` (much larger than it should be). In particular, this is bad when the sign of output for $i$ is positive. The model fixes this by repurposing another neuron to correct for the case when $j$ is present and $i$ is not. We omit the exact mechanism, but it takes advantage of the fact that the model has a ReLU at the very end, so it doesn't matter if output for a feature is very large and negative (the loss will be truncated at zero), but being large and positive is very bad.

Read the linked section of the Anthropic paper for details.

### Exercise - replicate the asymmetric superposition results

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵⚪⚪⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

Can you find a set of hyperparameters (importance, sparsity values, number of features and neurons) where this behaviour is observed?

Note - we recommend sticking with 5000 optimization steps or fewer. Overtraining this model can cause the magnitudes of $W_1$ to collapse, and $W_2$ to get very large, which makes the plot harder to visually interpret.

We've given you a utils function `plot_features_in_Nd_discrete` for you to visualize the results. It takes arguments `W1` and `W2` (for the two different weights matrices of your model) as well as `title` (a string) and `legend_names` (list of strings, which you can use to label the different importances within each instance). We've also given you some example code below - all you need to do is find the parameters and train the model.


<details>
<summary>Solution (set of values I found which produced this pattern)</summary>

I used `n_features=6` and `d_hidden=10` as seen in Anthropic's diagram. Feature probabilities are all $0.25$. Importances are the same as in the example case above; $I_i = 0.8^i$. Around half the instances I trained with these parameters had at least one monosemantic neuron *and* at least one pair of neurons which showed this pattern.

</details>



In [ ]:
# YOUR CODE HERE - replicate the asymmetric superposition results

See the [Colab solutions notebook](https://colab.research.google.com/drive/1mHKZpkhYAr0WWAQo2Y6pXL08yNfJHOVx?usp=sharing) for some sample code & outputs for these plots.

## Summary - what have we learned?

With toy models like this, it's important to make sure we take away generalizable lessons, rather than just details of the training setup.

The core things to take away form this paper are:

* What superposition is
* How it varies over feature importance and sparsity
* How it varies when we have correlated or anticorrelated features
* The difference between neuron and bottleneck superposition (or equivalently "computational and representational supervision")

# 4️⃣ Feature Geometry



> Note - this section is optional, since it goes into quite extreme detail about the specific problem setup we're using here. If you want, you can jump to the next section.


## Dimensionality

We've seen that superposition can allow a model to represent extra features, and that the number of extra features increases as we increase sparsity. In this section, we'll investigate this relationship in more detail, discovering an unexpected geometric story: features seem to organize themselves into geometric structures such as pentagons and tetrahedrons!

The code below runs a third experiment, with all importances the same. We're first interested in the number of features the model has learned to represent. This is well represented with the squared **Frobenius norm** of the weight matrix $W$, i.e. $||W||_F^2 = \sum_{ij}W_{ij}^2$.

<details>
<summary>Question - can you see why this is a good metric for the number of features represented?</summary>

By reordering the sums, we can show that the squared Frobenius norm is the sum of the squared norms of each of the 2D embedding vectors:

$$
\big\|W\big\|_F^2 = \sum_j \left(\sum_i W_{ij}^2\right) = \sum_{j}\big\|W_{[:, j]}\big\|^2
$$

Each of these embedding vectors has squared norm approximately $1$ if a feature is represented, and $0$ if it isn't. So this is roughly the total number of represented features.
</details>

If you run the code below, you'll also plot the total number of "dimensions per feature", $m/\big\|W\big\|_F^2$.


In [ ]:
cfg = Config(n_features=200, d_hidden=20, n_inst=20)

# For this experiment, use constant importance across features
feature_probability = 20 ** -t.linspace(0, 1, cfg.n_inst)

model = Model(
    cfg=cfg,
    device=device,
    feature_probability=feature_probability[:, None],
)
model.optimize(steps=10_000)

In [ ]:
utils.plot_feature_geometry(model)

Surprisingly, we find that this graph is "sticky" at $1$ and $1/2$. On inspection, the $1/2$ "sticky point" seems to correspond to a precise geometric arrangement where features come in "antipodal pairs", each being exactly the negative of the other, allowing two features to be packed into each hidden dimension. It appears that antipodal pairs are so effective that the model preferentially uses them over a wide range of the sparsity regime.

It turns out that antipodal pairs are just the tip of the iceberg. Hiding underneath this curve are a number of extremely specific geometric configurations of features.

How can we discover these geometric configurations? Consider the following metric, which the authors named the **dimensionality** of a feature:

$$
D_i = \frac{\big\|W_i\big\|^2}{\sum_{j} \big( \hat{W_i} \cdot W_j \big)^2}
$$

Intuitively, this is a measure of what "fraction of a dimension" a specific feature gets. Let's try and get a few intuitions for this metric:

* It's never less than zero.
    * It's equal to zero if and only if the vector is the zero vector, i.e. the feature isn't represented.
* It's never greater than one (because when $j = i$, the term in the denominator sum is equal to the numerator).
    * It's equal to one if and only if the $i$-th feature vector $W_i$ is orthogonal to all other features (because then $j=i$ is the only term in the denominator sum).
    * Intuitively, in this case the feature has an entire dimension to itself.
* If there are $k$ features which are all parallel to each other, and orthogonal to all others, then they "share" the dimensionality equally, i.e. $D_i = 1/k$ for each of them.
* The sum of all $D_i$ can't be greater than the total number of features $m$, with equality if and only if all the vectors are orthogonal.


### Exercise - compute dimensionality

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

Remember, $W$ has shape `(n_inst, d_hidden, n_features)`. The vectors $W_i$ refer to the feature vectors (i.e. they have length `d_hidden`), and you should broadcast your calculations over the `n_inst` dimension.

In [ ]:
@t.inference_mode()
def compute_dimensionality(W: Float[Tensor, "n_inst d_hidden n_features"]) -> Float[Tensor, "n_inst n_features"]:
    pass


tests.test_compute_dimensionality(compute_dimensionality)

The code below plots the fractions of dimensions, as a function of increasing levels of sparsity across our instances.

In [ ]:
W = model.W.detach()
dim_fracs = compute_dimensionality(W)

utils.plot_feature_geometry(model, dim_fracs=dim_fracs)

What's going on here? It turns out that the model likes to create specific weight geometries and kind of jumps between the different configurations. For example:

* With zero (or very small) sparsity, the feature basis isn't privileged by anything, and so the model represents features with arbitrary directions instead. There's no reason for some features to be represented faithfully and others not to be.
* When we get to higher levels of sparsity, the feature basis becomes privileged. So the model phase-transitions to representing some features in antipodal pairs, and the rest aren't interpreted.
* With further increases in sparsity, we transition to different geometries (see diagram below).

The moral? Superposition is very hard to pin down! There are many points between a dimensionality of 0 (not learning a feature) and 1 (dedicating a dimension to a feature). As an analogy, we often think of water as only having three phases: ice, water and steam. But this is a simplification: there are actually many phases of ice, often corresponding to different crystal structures (eg. hexagonal vs cubic ice). In a vaguely similar way, neural network features seem to also have many other phases within the general category of "superposition."

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/grid_all.png" width="900">

Note that we should take care not to read too much significance into these results. A lot of it depends delicately on the details of our experimental setup (e.g. we used $W^T W$, a positive semidefinite matrix, and there's a correspondence between low-dimensional symmetric pos-semidef matrices like these and the kinds of polytopes that we've seen in the plots above). But hopefully this has given you a sense of the relevant considerations when it comes to packing features into fewer dimensions.


# 5️⃣ Sparse Autoencoders in Toy Models

We now move on to sparse autoencoders, a recent line of work that has been explored by Anthropic in their [recent paper](https://transformer-circuits.pub/2023/monosemantic-features/index.html), and is currently one of the most interesting areas of research in mechanistic interpretability.

In the following set of exercises, you will:

- Build your own sparse autoencoder, writing its architecture & loss function,
- Train your SAE on the hidden activations of the `Model` class which you defined earlier (note the difference between this and the Anthropic paper's setup, since the latter trained SAEs on the MLP layer, whereas we're training it on a non-privileged basis),
- Extract the features from your SAE, and verify that these are the same as your model's learned features.

## Reading

You should read Anthropic's dictionary learning paper (linked above): the introduction and first section (problem setup) up to and including the "Sparse Autoencoder Setup" section. Make sure you can answer at least the following questions:

<details>
<summary>What is an autoencoder, and what is it trained to do?</summary>

Autoencoders are a type of neural network which learns efficient encodings / representations of unlabelled data. It is trained to compress the input in some way to a **latent representation**, then map it back into the original input space. It is trained by minimizing the reconstruction loss between the input and the reconstructed input.

The "encoding" part usually refers to the latent space being lower-dimensional than the input. However, that's not always the case, as we'll see with sparse autoencoders.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/sae-diagram.png" width="650">

</details>

<details>
<summary>Why is the hidden dimension of our autoencoder larger than the number of activations, when we train an SAE on an MLP layer?</summary>

As mentioned in the previous dropdown, usually the latent vector is a compressed representation of the input because it's lower-dimensional. However, it can still be a compressed representation even if it's higher dimensional, if we enforce a sparsity constraint on the latent vector (which in some sense reduces its effective dimensionality).

As for why we do this specifically for our autoencoder use case, it's because we're trying to recover features from superposition, in cases where there are **more features than neurons**. We're hoping our autoencoder learns an **overcomplete feature basis**.

</details>

<details>
<summary>Why does the L1 penalty encourage sparsity? (This isn't specifically mentioned in this paper, but it's an important thing to understand.)</summary>

Unlike $L_2$ penalties, the $L_1$ penalty actually pushes values towards zero. This is a well-known result in statistics, best illustrated below:

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/l1-viz.png" width="450">

See [this Google ML page](https://developers.google.com/machine-learning/crash-course/regularization-for-sparsity/l1-regularization) for more of an explanation (it also has a nice out-of-context animation!).

</details>

## Problem setup

Recall the setup of our previous model:

$$
\begin{aligned}
h &= W x \\
x' &= \operatorname{ReLU}(W^T h + b)
\end{aligned}
$$

We're going to train our autoencoder to just take in the hidden state activations $h$, map them to a larger (overcomplete) hidden state $z$, then reconstruct the original hidden state $h$ from $z$.

$$
\begin{aligned}
z &= \operatorname{ReLU}(W_{enc}(h - b_{dec}) + b_{enc}) \\
h' &= W_{dec}z + b_{dec}
\end{aligned}
$$

Note the choice to have a different encoder and decoder weight matrix, rather than having them tied - we'll discuss this more later.

It's important not to get confused between the autoencoder and model's notation. Remember - the model takes in features $x$, maps them to **lower-dimensional** vectors $h$, and then reconstructs them as $x'$. The autoencoder takes in these hidden states $h$, maps them to a **higher-dimensional but sparse** vector $z$, and then reconstructs them as $h'$. Our hope is that the elements of $z$ correspond to the features of $x$.

Another note - the use of $b_{dec}$ here might seem weird, since we're subtracting it at the start then adding it back at the end. The way we're treating this term is as a **centralizing term for the hidden states**. It subtracts some learned mean vector from them so that $W_{enc}$ can act on centralized vectors, and then this term gets added back to the reconstructed hidden states at the end of the model.

### Notation

The autoencoder's hidden activations go by many names. Sometimes they're called **neurons** (since they do have an activation function applied to them which makes them a privileged basis, like the neurons in an MLP layer). Sometimes they're called **features**, since the idea with SAEs is that these hidden activations are meant to refer to specific features in the data. However, the word feature is a bit [overloaded](https://www.lesswrong.com/posts/9Nkb389gidsozY9Tf/lewis-smith-s-shortform#fd64ALuWK8rXdLKz6) - ideally we want to use "feature" to refer to the attributes of the data itself - if our SAE's weights are randomly initialized, is it fair to call this a feature?!

For this reason, we'll be referring to the autoencoder's hidden activations as **SAE latents**. However, it's worth noting that people sometimes use "SAE features" or "neurons" instead, so try not to get confused (e.g. often people use "neuron resampling" to refer to the resampling of the weights in the SAE).

The new notation we'll adopt in this section is:

- `d_sae`, which is the number of activations in the SAE's hidden layer (i.e. the latent dimension). Note that we want the SAE latents to correspond to the original data features, which is why we'll need `d_sae >= n_features` (usually we'll have equality in this section).
- `d_in`, which is the SAE input dimension. This is the same as `d_hidden` from the previous sections because the SAE is reconstructing the model's hidden activations, however calling it `d_hidden` in the context of an SAE would be confusing. Usually in this section, we'll have `d_in = d_hidden = 2`, so we can visualize the results.

<details>
<summary>Question - in the formulas above (in the "Problem setup" section), what are the shapes of x, x', z, h, and h' ?</summary>

Ignoring batch and instance dimensions:

- `x` and `x'` are vectors of shape `(n_features,)`
- `z` is a vector of shape `(d_sae,)`
- `h` and `h'` are vectors of shape `(d_in,)`, which is equal to `d_hidden` from previous sections

Including batch and instance dimensions, all shapes have extra leading dimensions `(batch_size, n_inst, d)`.

</details>

## SAE class

We've provided the `SAEConfig` class below. Its arguments are as follows (for now you only need to worry about the first four):

- `n_inst`, which means the same as it does in your `Model` class
- `d_in`, the input size to your SAE (equal to `d_hidden` of your `Model` class)
- `d_sae`, the SAE's latent dimension size
- `tied_weights`, which is a boolean determining whether your encoder and decoder weights are tied
- `l1_coeff`, which is used in your loss function
- `weight_normalize_eps` - whenever you normalize weights, you should add this quantity to the denominator
- `architecture` - this will specify training different architectures to the standard one

We've also given you the `SAE` class below. Your job over the next 4 exercises will be to fill in the `__init__`, `W_dec_normalized`, `generate_batch` and `forward` methods.

In [ ]:
@dataclass
class SAEConfig:
    n_inst: int
    d_in: int
    d_sae: int
    l1_coeff: float = 0.2
    weight_normalize_eps: float = 1e-8
    tied_weights: bool = False
    architecture: Literal["standard", "gated"] = "standard"


class SAE(nn.Module):
    W_enc: Float[Tensor, "inst d_in d_sae"]
    _W_dec: Float[Tensor, "inst d_sae d_in"] | None
    b_enc: Float[Tensor, "inst d_sae"]
    b_dec: Float[Tensor, "inst d_in"]

    def __init__(self, cfg: SAEConfig, model: Model) -> None:
        super(SAE, self).__init__()

        assert cfg.d_in == model.cfg.d_hidden, "Model's hidden dim doesn't match SAE input dim"
        self.cfg = cfg
        self.model = model.requires_grad_(False)

        # YOUR CODE HERE
        raise NotImplementedError()

        self.to(device)

    @property
    def W_dec(self) -> Float[Tensor, "inst d_sae d_in"]:
        return self._W_dec if self._W_dec is not None else self.W_enc.transpose(-1, -2)

    @property
    def W_dec_normalized(self) -> Float[Tensor, "inst d_sae d_in"]:
        """Returns decoder weights, normalized over the autoencoder input dimension."""
        # YOUR CODE HERE
        raise NotImplementedError()

    def generate_batch(self, batch_size: int) -> Float[Tensor, "batch inst d_in"]:
        """
        Generates a batch of hidden activations from our model.
        """
        # YOUR CODE HERE
        raise NotImplementedError()

    def forward(
        self,
        h: Float[Tensor, "batch inst d_in"],
    ) -> tuple[
        dict[str, Float[Tensor, "batch inst"]],
        Float[Tensor, ""],
        Float[Tensor, "batch inst d_sae"],
        Float[Tensor, "batch inst d_in"],
    ]:
        """
        Forward pass on the autoencoder.

        Args:
            h: hidden layer activations of model

        Returns:
            loss_dict: dict of different loss function term values, for every (batch elem, instance)
            loss: scalar total loss (summed over instances & averaged over batch dim)
            acts: autoencoder feature activations
            h_reconstructed: reconstructed autoencoder input
        """
        # YOUR CODE HERE
        raise NotImplementedError()

    def optimize(
        self,
        batch_size: int = 1024,
        steps: int = 10_000,
        log_freq: int = 50,
        lr: float = 1e-3,
        lr_scale: Callable[[int, int], float] = constant_lr,
        resample_method: Literal["simple", "advanced", None] = None,
        resample_freq: int = 2500,
        resample_window: int = 500,
        resample_scale: float = 0.5,
    ) -> dict[str, list]:
        """
        Optimizes the autoencoder using the given hyperparameters.

        Args:
            model:              we reconstruct features from model's hidden activations
            batch_size:         size of batches we pass through model & train autoencoder on
            steps:              number of optimization steps
            log_freq:           number of optimization steps between logging
            lr:                 learning rate
            lr_scale:           learning rate scaling function
            resample_method:    method for resampling dead latents
            resample_freq:      number of optimization steps between resampling dead latents
            resample_window:    number of steps needed for us to classify a neuron as dead
            resample_scale:     scale factor for resampled neurons

        Returns:
            data_log:               dictionary containing data we'll use for visualization
        """
        assert resample_window <= resample_freq

        optimizer = t.optim.Adam(list(self.parameters()), lr=lr, betas=(0.0, 0.999))
        frac_active_list = []
        progress_bar = tqdm(range(steps))

        # Create lists to store data we'll eventually be plotting
        data_log = {"steps": [], "W_enc": [], "W_dec": [], "frac_active": []}

        for step in progress_bar:
            # Resample dead latents
            if (resample_method is not None) and ((step + 1) % resample_freq == 0):
                frac_active_in_window = t.stack(frac_active_list[-resample_window:], dim=0)
                if resample_method == "simple":
                    self.resample_simple(frac_active_in_window, resample_scale)
                elif resample_method == "advanced":
                    self.resample_advanced(frac_active_in_window, resample_scale, batch_size)

            # Update learning rate
            step_lr = lr * lr_scale(step, steps)
            for group in optimizer.param_groups:
                group["lr"] = step_lr

            # Get a batch of hidden activations from the model
            with t.inference_mode():
                h = self.generate_batch(batch_size)

            # Optimize
            loss_dict, loss, acts, _ = self.forward(h)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Normalize decoder weights by modifying them inplace (if not using tied weights)
            if not self.cfg.tied_weights:
                self.W_dec.data = self.W_dec_normalized

            # Calculate the mean sparsities over batch dim for each feature
            frac_active = (acts.abs() > 1e-8).float().mean(0)
            frac_active_list.append(frac_active)

            # Display progress bar, and append new values for plotting
            if step % log_freq == 0 or (step + 1 == steps):
                progress_bar.set_postfix(
                    lr=step_lr,
                    frac_active=frac_active.mean().item(),
                    **{k: v.mean(0).sum().item() for k, v in loss_dict.items()},  # type: ignore
                )
                data_log["W_enc"].append(self.W_enc.detach().cpu().clone())
                data_log["W_dec"].append(self.W_dec.detach().cpu().clone())
                data_log["frac_active"].append(frac_active.detach().cpu().clone())
                data_log["steps"].append(step)

        return data_log

    @t.no_grad()
    def resample_simple(
        self,
        frac_active_in_window: Float[Tensor, "window inst d_sae"],
        resample_scale: float,
    ) -> None:
        """
        Resamples dead latents, by modifying the model's weights and biases inplace.

        Resampling method is:
            - For each dead neuron, generate a random vector of size (d_in,), and normalize these vectors
            - Set new values of W_dec and W_enc to be these normalized vectors, at each dead neuron
            - Set b_enc to be zero, at each dead neuron
        """
        raise NotImplementedError()

    @t.no_grad()
    def resample_advanced(
        self,
        frac_active_in_window: Float[Tensor, "window inst d_sae"],
        resample_scale: float,
        batch_size: int,
    ) -> None:
        """
        Resamples latents that have been dead for 'dead_feature_window' steps, according to `frac_active`.

        Resampling method is:
            - Compute the L2 reconstruction loss produced from the hidden state vectors `h`
            - Randomly choose values of `h` with probability proportional to their reconstruction loss
            - Set new values of W_dec and W_enc to be these (centered and normalized) vectors, at each dead neuron
            - Set b_enc to be zero, at each dead neuron
        """
        raise NotImplementedError()

### Exercise - implement `__init__`

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-15 minutes on this exercise.
```

You should implement the `__init__` method below. This should define the weights `b_enc`, `b_dec`, `W_enc` and `_W_dec`. Use [Kaiming uniform](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_uniform_) for weight initialization, and initialize the biases at zero.

Note, we use `_W_dec` to handle the case of tied weights: it should be `None` if we have tied weights, and a proper parameter if we don't have tied weights. You can see how the `W_dec` property will return the correct decoder weights in both cases (either the decoder if it exists, or the transposed encoder).

<details>
<summary>Why might we want / not want to tie our weights?</summary>

In our `Model` implementations, we used a weight and its transpose. You might think it also makes sense to have the encoder and decoder weights be transposed copies of each other, since intuitively both the encoder and decoder's latent vectors meant to represent some feature's "direction in the original model's hidden dimension".

The reason we might not want to tie weights is pretty subtle. The job of the encoder is in some sense to recover features from superposition, whereas the job of the decoder is just to represent that feature faithfully if present. The diagram below might help illustrate this concept. This is why, when we have untied weights, we generally consider `W_dec` to be the "true direction" of the feature.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/w-dec-explained.png" width="700">

However, in these exercises we'll generally use untied weights.

</details>

Also, note that we've defined `self.cfg` and `self.model` for you in the init function - in the latter case, we've frozen the model's weights, because when you train your SAE you don't want to track gradients in your base model.

In [ ]:
tests.test_sae_init(SAE)

### Exercise - implement `W_dec_normalized`

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend 5-10 minutes on this exercise.
```

You should now fill in the `W_dec_normalized` property. Note that the existence of the `W_dec` property means you can safety refer to this attribute, without having to worry about `_W_dec` any more.

In [ ]:
tests.test_sae_W_dec_normalized(SAE)

### Exercise - implement `generate_batch`

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend 5-15 minutes on this exercise.
```

As mentioned, our data no longer comes directly from `Model.generate_batch`. Instead, we use `Model.generate_batch` to get our model input $x$, and then apply our model's `W` matrix to get its hidden activations $h=Wx$. Note that we're working with the model from the "Superposition in a Nonprivileged Basis" model, meaning there's no ReLU function to apply to get $h$.

You should fill in the `generate_batch` method now, then run the test. Note - remember to use `self.model` rather than `model`!

In [ ]:
tests.test_sae_generate_batch(SAE)

### Exercise - implement `forward`

```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵🔵

You should spend up to 20-30 minutes on this exercise.
```

You should calculate the autoencoder's hidden state activations as $z = \operatorname{ReLU}(W_{enc}(h - b_{dec}) + b_{enc})$, and then reconstruct the output as $h' = W_{dec}z + b_{dec}$. A few notes:

- The **first variable** we return is a `loss_dict`, which contains the loss tensors of shape `(batch_size, n_inst)` for both terms in our loss function (before multiplying by the L1 coefficient). This is used for logging, and it'll also be used later in our neuron resampling methods. For this architecture, your keys should be `"L_reconstruction"` and `"L_sparsity"`.
    - Important note for the loss terms - the reconstruction loss is the squared difference between input & output **averaged** over the `d_in` dimension, but the sparsity penalty is the L1 norm of the hidden activations **summed** over the `d_sae` dimension. Can you see why we average one but sum the other?

<details>
<summary>Answer - why we average L2 loss over <code>d_in</code> but sum L1 loss over <code>d_sae</code></summary>

Imagine if we doubled the latent dimension, but kept everything the same. The gradient applied to any given latent vector from the reconstruction loss would still be the same, but the gradient from the sparsity penalty would have doubled (because we're averaging it over a larger number of features). This means that in the limit, the sparsity penalty wouldn't matter at all, and the only important thing would be getting zero reconstruction loss.

</details>

- The **second variable** we return is the scalar `loss` term, which is an aggregate of the losses in `loss_dict`, summed over instances and averaged over the batch dimension. You should multiply the L1 loss by `cfg.l1_coef` in this calculation.
- The **third variable** we return is the hidden state activations `acts`, which are also used later for neuron resampling (as well as logging how many latents are active).
- The **fourth variable** we return is the reconstructed hidden states `h_reconstructed`, i.e. the autoencoder's actual output.

In [ ]:
tests.test_sae_forward(SAE)

## Training your autoencoder

The `optimize` method has been given to you. A few notes on how it differs from your previous model:

- Before each optimization step, we implement **neuron resampling** - we'll get to this later.
- We have more logging, via the `data_log` dictionary - we'll use this for visualization.
- We've used `betas=(0.0, 0.999)`, to match the description in [Anthropic's Feb 2024 update](https://transformer-circuits.pub/2024/feb-update/index.html#dict-learning-loss) - although they document it to work better specifically for large models, we may as well match it here.

We'll now train our base model (doing all the normal visualizations to make sure it's working okay), then train our SAE to reconstruct its activations. We'll be using a feature probability of 1% for the rest of this section.

First, the model (with weights & hidden activations visualized):

In [ ]:
d_hidden = d_in = 2
n_features = d_sae = 5
n_inst = 8

cfg = Config(n_inst=n_inst, n_features=n_features, d_hidden=d_hidden)

model = Model(cfg=cfg, device=device)
model.optimize(steps=10_000)

sae = SAE(cfg=SAEConfig(n_inst=n_inst, d_in=d_in, d_sae=d_sae), model=model)

h = sae.generate_batch(500)

In [ ]:
utils.plot_features_in_2d(model.W, title="Base model")

In [ ]:
utils.plot_features_in_2d(
    einops.rearrange(h, "batch inst d_in -> inst d_in batch"),
    title="Hidden state representation of a random batch of data",
)

Now, let's optimize! We've also created a function `animate_features_in_2d` which can visualize the training over time. The code below will save the visualization, you may have to download it to open it in your browser.

In [ ]:
data_log = sae.optimize(steps=25_000)

utils.animate_features_in_2d(
    {
        "Encoder weights": t.stack(data_log["W_enc"]),
        "Decoder weights": t.stack(data_log["W_dec"]).transpose(-1, -2),
    },
    steps=data_log["steps"],
    filename="animation-training.html",
    title="SAE on toy model",
)

In [ ]:
with open(section_dir / "animation-training.html") as f:
    display(HTML(f.read()))

In other words, the autoencoder is generally successful at discovering the model's features, and maybe somtimes it's even lucky enough to learn all 5, but most of the time it learns **"dead features"** which never activate. You can check this by graphing the feature probabilities over training, in the code below. You should find that there are 2 types of features: ones which converge to the expected feature probability of $0.01$, and ones which quickly converge to zero. This is an annoying problem when it comes to training sparse autoencoders.

In [ ]:
utils.frac_active_line_plot(
    frac_active=t.stack(data_log["frac_active"]),
    title="Probability of sae features being active during training",
    avg_window=10,
)

## Resampling

From Anthropic's paper (replacing terminology "dead neurons" with "dead latents" in accordance with how we're using the term):

> Second, we found that over the course of training some latents cease to activate, even across a large number of datapoints. We found that “resampling” these dead latents during training gave better results by allowing the model to represent more features for a given autoencoder hidden layer dimension. Our resampling procedure is detailed in [Autoencoder Resampling](https://transformer-circuits.pub/2023/monosemantic-features/index.html#appendix-autoencoder-resampling), but in brief we periodically check for latents which have not fired in a significant number of steps and reset the encoder weights on the dead latents to match data points that the autoencoder does not currently represent well.

Your next task is to implement this resampling procedure.

### Exercise - implement `resample_simple`

```c
Difficulty: 🔴🔴🔴🔴⚪
Importance: 🔵🔵🔵🔵⚪

You should spend up to 20-30 minutes on this exercise.
```

The process Anthropic describes for resampling SAE latents is pretty involved, so we'll start by implementing a simpler version of it. Specifically, we'll implement the following algorithm for each instance `inst`:

* Find all the dead latents (i.e. the values `(inst, d)` where `frac_active_in_window[:, inst, d]` are all zero).
* For each of these, do the following:
    * Generate a new random vector `v` of length `d_in`.
    * Set the decoder weights `W_dec[inst, d, :]` to this new vector `v`, normalized.
    * Set the encoder weights `W_enc[inst, :, d]` to this new vector `v`, scaled to have norm `resample_scale`.
    * Set the encoder biases `b_enc[inst, d]` to zero.

The test function we've given you will check that your function replaces / zeros the correct weights.

In [ ]:
tests.test_resample_simple(SAE)

Once you've passed the tests, train your model again, and watch the animation to see how the neuron resampling has helped the training process. You should be able to see the resampled neurons in red.

In [ ]:
sae = SAE(cfg=SAEConfig(n_inst=n_inst, d_in=d_in, d_sae=d_sae), model=model)

data_log = sae.optimize(steps=25_000, resample_method="simple")

utils.animate_features_in_2d(
    {
        "Encoder weights": t.stack(data_log["W_enc"]),
        "Decoder weights": t.stack(data_log["W_dec"]).transpose(-1, -2),
    },
    steps=data_log["steps"],
    filename="animation-resampling.html",
    title="SAE on toy model with simple resampling",
)

In [ ]:
with open(section_dir / "animation-resampling.html") as f:
    display(HTML(f.read()))

Much better!

Now that we have pretty much full reconstruction on our features, let's plot them! The `animate_features_in_2d` function also offers features to plot rows of data, and display values when you hover over them.

### Exercise - implement `resample_advanced`


```c
Difficulty: 🔴🔴🔴🔴🔴
Importance: 🔵🔵⚪⚪⚪

You should spend up to 20-40 minutes on this exercise.
```

This section can be considered optional if you've already implemented the simpler version of `resample` above. However, if you're interested in a version of it which hues close to [Anthropic's methodology](https://transformer-circuits.pub/2023/monosemantic-features/index.html#appendix-autoencoder-resampling), then you might still be interested in this exercise.

The main difference we'll make is in how the resampled values are chosen. Rather than just drawing them randomly from a distribution and normalizing them, we'll be **sampling them with replacement from a set of input activations $h$, with sampling probabilities weighted by the squared $L_2$ loss of the autoencoder on each input**. Intuitively, this will make it more likely that our resampled neurons will represent feature directions that the autoencoder is currently doing a bad job of representing.

The new resampling algorithm looks like the following - for each instance we:

* Generate a batch of hidden data `h` from your SAE and compute its squared reconstruction loss `l2_squared`. It should have shape `(batch_size, n_inst)`. If the L2 loss for this instance `l2_squared[:, inst]` is zero everywhere, we can skip this instance.
* Find the dead latents for this instance (i.e. the values `(inst, d)` where `frac_active_in_window[:, inst, d]` are all zero).
* For each of these, do the following:
    * Randomly sample a vector `v = h[x, inst, :]`, where `0 <= x < batch_size` is chosen according to the distribution with probabilities proportional to `l2_squared[:, inst]`.
    * Set the decoder weights `W_dec[inst, d, :]` to this new vector `v`, normalized.
    * Set the encoder weights `W_enc[inst, :, d]` to this new vector `v`, scaled to have norm `resample_scale * avg_W_enc_alive_norm` (where the term `avg_W_enc_alive_norm` is the mean norm of the encoder weights of alive neurons for that particular instance).
    * Set the encoder biases `b_enc[inst, d]` to zero.

So we really have just 2 changes: the added use of `avg_W_enc_alive_norm` for the encoder weights, and the sampling from the L2-based distribution to get our vectors `v`. Because this function can get a bit messy, we recommend you iterate through the instances rather than trying to resample them all at once.

For the sampling, we recommend that you use `torch.distributions.categorical.Categorical` to define a probability distribution, which can then be sampled from using the `sample` method. We've included an example of how to use this function below.

<details>
<summary>Example of using <code>Categorical</code>.</summary>

```python
from torch.distributions.categorical import Categorical

# Define a prob distn over (0, 1, 2, 3, 4) with probs proportional to (4, 3, 2, 1, 0)
values = t.arange(5).flip(0)
probs = values.float() / values.sum()
distribution = Categorical(probs = probs)

# Sample a single value from it
distribution.sample() # tensor(1)

# Sample multiple values with replacement (values will mostly be in the lower end of the range)
distribution.sample((10,)) # tensor([1, 1, 3, 0, 0, 1, 0, 3, 2, 2])
```

When you're sampling multiple times, make sure to pass a 1D tensor rather than a scalar.

</details>

Once you've implemented this resampling method, run the tests:

In [ ]:
tests.test_resample_advanced(SAE)

After passing the tests, you can try training & visualizing your SAE again. You might not spot a lot of improvement with this resampling method in 2 dimensions, but for much higher-dimensional spaces it becomes highly beneficial to resample neurons in a more targeted way.

### Exercise - overcomplete basis

```c
Difficulty: 🔴⚪⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 5-10 minutes on this exercise.
```

Try training your autoencoder with an overcomplete basis: `d_sae` strictly larger than `n_features`.

- Does your model learn the features faster?
- How long does it take for all features to be learned by at least one of the autoencoder neurons, and how long before all neurons are either dead or representing exactly one feature?
- Can you devise a procedure which kills neurons which are highly correlated with other neurons, and thereby allows the model learn a 1-1 correspondence between neurons and features, which is learned faster than when we used `d_sae = n_features`?

### Exercise - tied weights

```c
Difficulty: 🔴🔴⚪⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-15 minutes on this exercise.
```

One possible approach you can take is to tie your embedding and unembedding weights together, i.e. have `W_dec = W_enc.T` (for any given instance). Try rewriting your code to remove `W_dec` and replace all instances of `W_dec` with the transposed version of `W_enc`. You'll also need to rewrite the `normalize_decoder` method so that it operates on the encoder rather than decoder.

How do your results change? Why do you think this is, and what do you think the justifications are for tying or not tying the weights?

<details>
<summary>Answer (what you should see, and why)</summary>

You should see that the model is able to learn the features more quickly, maybe without resampling any neurons at all.

This is because our toy model is a pretty special case, where all the features have the same importance, are independent, and are evenly spaced around the unit circle, meaning there's no real difference between the notion of encoding and decoding directions (they're both equal to the direction in `model.W`). But this might not always be the case. As an example, consider the case where 2 of our 5 features are highly correlated. We would still want the decoder directions to be the same, because the purpose of the decoder is to reconstruct the original features. But the encoder is **optimized to detect interfering features in superposition**, so it might want to learn a slightly different set of directions (e.g. if features are correlated, or they have different importances).

</details>

### Exercise - explore untied weights

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to 10-25 minutes on this exercise.
```

In the previous exercises, our model has learned a uniform solution. That is, all represented features are equally spaced around the unit circle, in the 2D hidden dimension. This guarantees that the encoder and decoder weights will be the same (we can argue this by symmetry). But intuitively, we shouldn't expect this to happen all the time. For an idea of why, read [Neel's public comment document](https://docs.google.com/document/u/0/d/187jfZSbhRjjQaazjYlThBsKp3Q0Pw3VdIHVST9H2dvw/mobilebasic) on the dictionary learning paper. TL;DR - the decoder just tries to minimize reconstruction loss so it should represent the features faithfully, but the encoder has the added job of disentangling features which might be in interference with each other.

Below is some code to create a model, and manually alter its weights so that two features have high correlation. What do you think will happen when you train an autoencoder on this model? What will happen to the encoder weights? How about the decoder weights?

In [ ]:
cfg = Config(n_inst=8, n_features=4, d_hidden=2)

model = Model(cfg=cfg, device=device, feature_probability=0.025)

# Replace the model's weights with a custom-chosen non-uniform set of features
angles = 2 * t.pi * t.tensor([0.0, 0.25, 0.55, 0.70])
angles = angles + t.rand((cfg.n_inst, 1))
model.W.data = t.stack([t.cos(angles), t.sin(angles)], dim=1).to(device)

utils.plot_features_in_2d(
    model.W,
    title=f"Superposition: {cfg.n_features} features in 2D space (non-uniform)",
    subplot_titles=[f"Instance #{i}" for i in range(1, 1 + cfg.n_inst)],
)

## Gated & JumpReLU SAEs



There are many different SAE architecture variants being explored at the moment. One especially exciting one is the **Gated SAE**, described in detail in this paper from [DeepMind](https://arxiv.org/pdf/2404.16014). We can motivate this architecture by starting with two observations

1. **Empirically, features usually seem to want to be binary.** For instance, we often see features like "is this about a basketball" which are better thought of as "off" or "on" than occupying some continuous range from 0 to 1. In practice reconstructing the precise coefficients does matter, and they often seem important for indicating something like the model's confidence in a particular feature being present. But still, we'd ideally like an architecture which can learn this discontinuity.

One easy option would be to have a discontinuous activation function in the hidden layer of our SAE, such as a **Jump ReLU**. This activation has a jump at some value $\theta$, and could allow us to represent this nonlinearity.

<img src="https://res.cloudinary.com/lesswrong-2-0/image/upload/f_auto,q_auto/v1/mirroredImages/wZqqQysfLrt2CFx4T/zzrdot3xexvcz3mqghn8" width="240">

However, there's another problem which Jump ReLUs won't solve:

2. **SAEs suffer from [shrinkage](https://www.alignmentforum.org/posts/3JuSjTZyMzaSeTxKk/addressing-feature-suppression-in-saes). Recall that the actual objective we want is that the L0 "norm" (the number of non-zero elements) of the hidden layer is small, and we use the L1 norm as a proxy for this. The two loss term in the SAE loss function have conflicting goals: the reconstruction term wants to make the autoencoder good at reconstructing the input, and the sparsity term wants to shrink the magnitude of the hidden layer. This means that even when perfect reconstruction is possible with only a single hidden unit activated, the sparsity loss will bias the magnitude of this hidden unit to zero, and the reconstruction will be worse.  

This brings us to **Gated SAEs**, which seem to fix both problems by having a Heaviside term which applies a discontinuity, and decoupling this term from the magnitude term. Instead of our standard function for computing SAE activations:

$$
\mathbf{f}(\mathbf{x}):=\operatorname{ReLU}\left(\mathbf{W}_{\mathrm{enc}}\left(\mathbf{x}-\mathbf{b}_{\mathrm{dec}}\right)+\mathbf{b}_{\mathrm{enc}}\right)
$$

we instead use:

$$
\tilde{\mathbf{f}}(\mathbf{x}):=\underbrace{\mathbf{1} \overbrace{\left[\mathbf{W}_{\text {gate }}\left(\mathbf{x}-\mathbf{b}_{\text {dec }}\right)+\mathbf{b}_{\text {gate }}\right)}^{\pi_{\text {gate }}(\mathbf{x})}>0]}_{\mathbf{f}_{\text {gate }}(\mathbf{x})} \odot \underbrace{\operatorname{ReLU}\left(\mathbf{W}_{\text {mag }}\left(\mathbf{x}-\mathbf{b}_{\text {dec }}\right)+\mathbf{b}_{\text {mag }}\right)}_{\mathbf{f}_{\text {mag }}(\mathbf{x})},
$$
where $\mathbf{1}[\cdot > 0]$ is the pointwise Heavisde step function and $\odot$ is elementwise multiplication. The features' gate and activation magnitudes are computed by weight matrices, $W_{\text{mag}}$ and $W_{\text{gate}}$. Interestingly, if we tie the gated and magnitude weights as $\left(\mathbf{W}_{\text {mag }}\right)_{i j}:=\left(\exp \left(\mathbf{r}_{\text {mag }}\right)\right)_i \cdot\left(\mathbf{W}_{\text {gate }}\right)_{i j}$, then we can show that this is basically equivalent to a Jump ReLU activation function with a parameterized threshold value $\theta$ (left as an exercise to the reader!).

You might be wondering, how can we train this SAE? Ideally we'd place a sparsity penalty on the term $f_{\text{gate}}(\mathbf{x})$, since that's the thing which determines whether our activations will be zero or not. Unfortunately we can't do that, because gradients won't propagate through the Heaviside function (it's discontinuous). Instead, we apply a sparsity penalty to the preactivation $\pi_{\text {gate }}(\mathbf{x})$. Lastly, to ensure $f_{\text{gate}}$  aids reconstruction by detecting active features, we add an auxiliary task requiring that these same rectified preactivations can be used by the decoder to produce a good reconstruction. This gives us our loss function:

$$
\mathcal{L}_{\text {gated }}(\mathbf{x}):=\underbrace{\|\mathbf{x}-\hat{\mathbf{x}}(\tilde{\mathbf{f}}(\mathbf{x}))\|_2^2}_{\mathcal{L}_{\text {reconstruct }}}+\underbrace{\lambda\left\|\operatorname{ReLU}\left(\boldsymbol{\pi}_{\text {gate }}(\mathbf{x})\right)\right\|_1}_{\mathcal{L}_{\text {sparsity }}}+\underbrace{\left\|\mathbf{x}-\hat{\mathbf{x}}_{\text {frozen }}\left(\operatorname{ReLU}\left(\boldsymbol{\pi}_{\text {gate }}(\mathbf{x})\right)\right)\right\|_2^2}_{\mathcal{L}_{\text {aux }}}
$$

### Exercise - implement gated SAE

```c
Difficulty: 🔴🔴🔴🔴🔴
Importance: 🔵🔵🔵🔵⚪

You should spend up to 60 minutes on this exercise.
```

Now, you have all the information you need to implement a Gated SAE and compare it to the standard model. The solutions file contains a full implementation of the Gated model. Here's some guidance:

- We recommend you tie the gate and magnitude weights by default, as $\left(\mathbf{W}_{\text {mag }}\right)_{i j}:=\left(\exp \left(\mathbf{r}_{\text {mag }}\right)\right)_i \cdot\left(\mathbf{W}_{\text {gate }}\right)_{i j}$ like they do in the paper. This kind of tying is arguably a lot less unnatural than tying encoder & decoder weights. If you're *also* tying weights, then you can interpret that as $W_{\text{dec}} = W_{\text{gate}}$.

<details>
<summary>Help - I'm not sure how I should implement this weight tying.</summary>

We recommend using a property, like this:

```python
@property
def W_mag(self) -> Float[Tensor, "inst d_in d_sae"]:
    assert self.cfg.architecture == "gated", "W_mag only available for gated model"
    return self.r_mag.exp().unsqueeze(1) * self.W_gate
```

Then you only have to define `r_mag` and `W_gate`. Note, this means you should be careful when you're resampling, because you can't set the values of `W_mag` directly.

</details>

- For the forward pass and the loss function, you can reference Appendix G in the [DeepMind paper](https://arxiv.org/pdf/2404.16014), on page 34. We recommend sticking to the naming convention used by that appendix, as you'll probably find this easiest.
- Remember to create _and resample_ different weights if you're using the Gated architecture. For instance, if Gated then you should be zeroing `b_mag`, `b_gate` and `r_mag` at all dead latents.


In [ ]:
# See Solutions Colab for implementation of gated model (or solutions file in repo, which has option for
# implementing standard or gated, according to the sae config)

### Exercise - demonstrate advantage of Gated models

```c
Difficulty: 🔴🔴🔴⚪⚪
Importance: 🔵🔵🔵⚪⚪

You should spend up to ~30 minutes on this exercise.
```

If you implemented the Gated architecture correctly, you might not have found much advantage over the standard architecture. This is because we're still using features which are uniform in the range [0, 1], and so a large part of the reason we believe Gated to be better than standard just isn't true.

You can construct a case where the Gated model outperforms the standard model, as described by the [DeepMind paper](https://arxiv.org/pdf/2404.16014) in appendix F (they describe it in terms of Jump ReLU outperforming base models, but this works for Gated models too for the same reason). You'll need to return all the way back to the `Model.generate_batch` method at the start of our exercises. If `cfg.feat_mag_distn = "jump"`, then you should have a different distribution for feature magnitudes (we suggest choosing values $\mu$, $\sigma$ so that the feature magnitudes are still almost always in the range $[0, 1]$, but that the mode of the distribution is larger than zero).

If you do this correctly, then you should be able to generate visualizations like this one (code below). The first row are the model's hidden activations, the second row is the standard model's reconstruction, and the third is the gated model's reconstruction. You should be able to see when you hover over points in the first row (especially those near zero) that the standard model just maps these to zero, but the Gated model is able to separate them. Note that you might also see an improvement in the Gated model when you use the uniform distribution, but it'll probably be more pronounced with a distribution that has a clearer discontinuity.

In [ ]:
# YOUR CODE HERE - train gated & standard models, and create animation to compare hidden reconstructions. Code to produce
# the animation is given below, assuming `h`, `h_r_standard` and `h_r_gated` each have shape (batch_size, instances, d_in).

rearrange_h = lambda h: einops.rearrange(h, "b i d_in -> i d_in b")

utils.animate_features_in_2d(
    {
        "Hidden states": rearrange_h(h),
        "Reconstructions (standard)": rearrange_h(h_r_standard),
        "Reconstructions (gated)": rearrange_h(h_r_gated),
    },
    filename="animation-reconstructions-standard-and-gated.html",
    title="Hidden state vs reconstructions",
)

In [ ]:
with open(section_dir / "animation-reconstructions-standard-and-gated.html") as f:
    display(HTML(f.read()))

The plot to have in your head is the right hand side one below, from DeepMind's appendix (which should be fairly easy to replicate, if you're replicating the results of the animation). The left histogram shows the distribution along a particular feature direction - the blue represents the distribution from interference when the feature is off but other non-orthogonal features are on, and the red represents the distribution then the feature is on. The distributions form a clearly bimodal pattern, and we can see in the figure on the right how a jump discontinuity (like the one provided by ReLU or by Gated models) can better model this discontinuity, by correctly reconstructing more of the interference cases (blue) as zero.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/distn-gated.png" width="900">

# 6️⃣ Bonus

## Suggested paper replications



### [Toy Models of Superposition](https://transformer-circuits.pub/2022/toy_model/index.html#phase-change)

There are several aspects of this paper which we didn't cover in these exercises. In particular, **superposition as a phase change** studies the interaction between sparsity and relative feature importance, and finds a phase change in the optimal weight configuration as these inputs are varied. Some examples can be found in [this notebook](https://github.com/wattenberg/superposition/blob/main/Exploring_Exact_Toy_Models.ipynb)

This might be a good replication for you if:

* You enjoy diving into the specific mathematical details of superposition
* You find theoretical work interesting, as well as empirical work
* You've enjoyed the first ~3 exercise sets in this section

<br>



### [Polysemanticity and Capacity in Neural Networks](https://arxiv.org/pdf/2210.01892.pdf)

This is a paper by Redwood Research, which builds on the ideas we discussed in the first three four of this paper (toy models of superposition, and the results on feature geometry).

They deeply study a measure called capacity, which is the same as what we called dimensionality above. Their results suggest an explanation for why features are often sharply "pinned" to either 0 or 1 capacity (i.e. not represented at all, or represented orthogonally to all other features).

This might be a good replication for you if:

* You enjoy diving into the specific mathematical details of superposition
* You're comfortable with mathematical topics like linear algebra and calculus
* You find theoretical work interesting, as well as empirical work
* You've enjoyed the first ~4 exercise sets in this section

<!-- ### [Finding Neurons in a Haystack: Case Studies with Sparse Probing](https://arxiv.org/abs/2305.01610)

The authors train a set of sparse linear probes on neuron activations to predict the presence of certain input features. They manage to find **sparse combinations of neurons which represent many features in superposition**, e.g. a neuron which activates on the bigram phrase "social security" but not either word individually (see image below).

Note that this paper is slightly less relevant now that dictionary learning with SAEs has superceded its methodology - but it still represents a large step forward for the goal of extracting features from superposition in MLPs.

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/socialsecurity.png" width="750"> -->


### [Superposition, Memorization and Deep Double Descent](https://transformer-circuits.pub/2023/toy-double-descent/index.html)

*Note - this is more of a guided replication than the other two! It was originally its own set of exercises before being moved to the bonus section*.

For this suggested replication, we'll look at the Anthropic paper on Double Descent & superposition. This paper ties the phenomena of [double descent](https://openai.com/research/deep-double-descent) to models of superposition. The theory posed by this paper goes roughly as follows:

* Initially, the model learns a **memorising solution** where datapoints are represented in superposition. This doesn't generalize, so we get low training loss but high test loss.
* Later, the model learns a **generalizing solution** where features are learned and represented in superposition. This generalizes, so we get low training loss and low test loss.
* The spike in loss between these two happens when the model transitions between the memorising and generalizing solutions.

What does it mean to represent datapoints in superposition? If you've done the exercises on correlated / anticorrelated features in an earlier section, you'll know that anticorrelated features are easier to represent in superposition because they don't interfere with each other. This is especially true if features aren't just anticorrelated but are **mutually exclusive**. From the Anthropic paper:

> Consider the case of a language model which verbatim memorizes text. How can it do this? One naive idea is that it might use neurons to create a lookup table mapping sequences to arbitrary continuations. For every sequence of tokens it wishes to memorize, it could dedicate one neuron to detecting that sequence, and then implement arbitrary behavior when it fires. The problem with this approach is that it's extremely inefficient – but it seems like a perfect candidate for superposition, since each case is mutually exclusive and can't interfere.

We'll study this theory in the context of a toy model. Specifically, we'll use the toy model that we worked with in the first section of this paper, but we'll train it in a very particular way: by generating a random batch of data, and then using that same batch for the entire training process. We'll see what happens when the batch sizes change, but the number of features change. According to our theory, the model should represent datapoints in superposition when the batch size is smaller than the number of features, and it should represent features in superposition when the batch size is larger than the number of features.

Rather than giving you a set of exercises to complete, we're leaving this section open-ended. You should consider it more as a paper replication than a set of structured exercises. However, we will give you a few tips:

* Rather than using the Adam optimizer, the paper recommends AdamW, with a default weight decay of `WEIGHT_DECAY = 1e-2`.
    * Weight decay constrains the norm of weights, so that they don't grow too large. With no weight decay, we could in theory memorize an arbitrarily large number of datapoints and represent them evenly spaced around the unit circle; then we can perfectly reconstruct them as long as we have a large enough weight vector to project them onto.
* The paper recommends a learning rate consisting of a linear warmup up to `NUM_WARMUP_STEPS = 2500` (i.e. we increase the learning rate linearly from zero up to `LEARNING_RATE = 1e-3`), followed by cosine decay until the end of training at `NUM_BATCH_UPDATES = 50_000`.
* The paper recommends using a sparsity of 0.999 for the features, and 10,000 features total. However, we recommend instead using `SPARSITY = 0.99` and `N_FEATURES = 1000` (following the replication by Marius Hobbhahn). This will cause our model to train faster, while still observing fundamentally the same patterns.
* When generating the batch of data, you should normalize it (so each vector for a given batch index & instance has unit norm). The rest of the data generation process should be the same as in the first section of this notebook.
* Technically you only need one instance. However, we recommend using a few (e.g. 5-10) so you can pick the instance with lowest loss at the end of training. This is because (thanks to our best frend randomness) not all instances will necessarily learn the optimal solution. In our implementation (code below), we rewrite the `optimize` function to return `(batch_inst, W_inst)` at the end, where `batch_inst` is the batch which had the lowest loss by the end of training, and `W_inst` are the learned weights for that same instance. This is precisely the data you'll need to make the 2D feature plots featured in the paper.
* You can repurpose the function to calculate **dimensionality** from the section on feature geometry. See the paper for a discussion of a generalized dimensionality function, which doesn't just measure dimensionality of features, but also of datapoints.

To get you started, here are some constants which you might find useful:

In [ ]:
NUM_WARMUP_STEPS = 2500
NUM_BATCH_UPDATES = 50_000

WEIGHT_DECAY = 1e-2
LEARNING_RATE = 1e-3

BATCH_SIZES = [3, 5, 6, 8, 10, 15, 30, 50, 100, 200, 500, 1000, 2000]

N_FEATURES = 1000
N_INSTANCES = 5
N_HIDDEN = 2
SPARSITY = 0.99
FEATURE_PROBABILITY = 1 - SPARSITY

Also, if you want some help with the visualisation, the code below will produce the 2D feature visualisations like those found at the bottom of [this figure](https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/fig-2d.png), for all batch sizes stacked horizontally, assuming:

* `features_list` is a list of detached `W`-matrices for single instances, i.e. each has shape `(2, n_features)` (these will be used to produce the blue plots on the first row)
* `data_list` is a list of the projections of our batch of data onto the hidden directions of that same instance, i.e. each has shape `(2, batch_size)` (these will be used to produce the red plots on the second row)

A demonstration is given below (the values are meaningless, they've just been randomly generated to illustrate how to use this function).

In [ ]:
features_list = [t.randn(size=(2, 100)) for _ in BATCH_SIZES]
hidden_representations_list = [t.randn(size=(2, batch_size)) for batch_size in BATCH_SIZES]

utils.plot_features_in_2d(
    features_list + hidden_representations_list,
    colors = [["blue"] for _ in range(len(BATCH_SIZES))] + [["red"] for _ in range(len(BATCH_SIZES))],
    title = "Double Descent & Superposition (num features = 100)",
    subplot_titles = [f"Features (batch={bs})" for bs in BATCH_SIZES] + [f"Data (batch={bs})" for bs in BATCH_SIZES],
    allow_different_limits_across_subplots = True,
    n_rows = 2,
)

You can click on the dropdown below to see a full replication of these results (or just look at bits of this, to get some hints).

<details>
<summary>Implementation</summary>

Some basic setup code:

```python
import math
from typing import Any
import pandas as pd
import plotly.express as px

NUM_WARMUP_STEPS = 2500
NUM_BATCH_UPDATES = 50_000
# EVAL_N_DATAPOINTS = 1_000

WEIGHT_DECAY = 1e-2
LEARNING_RATE = 1e-3

BATCH_SIZES = [3, 4, 5, 6, 8, 10, 15, 20, 30, 50, 100, 200, 300, 500, 1000, 2000, 3000]
# SMALLER_BATCH_SIZES = [3, 6, 10, 30, 100, 500, 2000]

N_FEATURES = 1000
N_INSTANCES = 10
N_HIDDEN = 2
SPARSITY = 0.99
FEATURE_PROBABILITY = 1 - SPARSITY
```

Our new schedulers, in line with Anthropic's writeup:

```python
def linear_warmup_lr(step, steps):
    """Increases linearly from 0 to 1."""
    return step / steps

def anthropic_lr(step, steps):
    """As per the description in the paper: 2500 step linear warmup, followed by cosine decay to zero."""
    if step < NUM_WARMUP_STEPS:
        return linear_warmup_lr(step, NUM_WARMUP_STEPS)
    else:
        return cosine_decay_lr(step - NUM_WARMUP_STEPS, steps - NUM_WARMUP_STEPS)
```

Our new model, with some of its methods rewritten:

```python
class DoubleDescentModel(Model):
    W: Float[Tensor, "inst d_hidden feats"]
    b_final: Float[Tensor, "inst feats"]
    # Our linear map (for a single instance) is x -> ReLU(W.T @ W @ x + b_final)

    @classmethod
    def dimensionality(
        cls, data: Float[Tensor, "... batch d_hidden"]
    ) -> Float[Tensor, "... batch"]:
        """
        Calculates dimensionalities of data. Assumes data is of shape ... batch d_hidden, i.e. if it's 2D then
        it's a batch of vectors of length `d_hidden` and we return the dimensionality as a 1D tensor of length
        `batch`. If it has more dimensions at the start, we assume this means separate calculations for each
        of these dimensions (i.e. they are independent batches of vectors).
        """
        # Compute the norms of each vector (this will be the numerator)
        squared_norms = einops.reduce(data.pow(2), "... batch d_hidden -> ... batch", "sum")
        # Compute the denominator (i.e. get the dotproduct then sum over j)
        data_normed = data / data.norm(dim=-1, keepdim=True)
        interference = einops.einsum(
            data_normed, data, "... batch_i d_hidden, ... batch_j d_hidden -> ... batch_i batch_j"
        )
        polysemanticity = einops.reduce(
            interference.pow(2), "... batch_i batch_j -> ... batch_i", "sum"
        )
        assert squared_norms.shape == polysemanticity.shape

        return squared_norms / polysemanticity

    def generate_batch(self, batch_size: int) -> Float[Tensor, "batch inst feats"]:
        """
        New function for generating batch, so we can normalize it.
        """
        # Get batch from prev method
        batch = super().generate_batch(batch_size)

        # Normalize the batch (i.e. so each vector for a particular batch & instance has norm 1)
        # (need to be careful about vectors with norm zero)
        norms = batch.norm(dim=-1, keepdim=True)
        norms = t.where(norms.abs() < 1e-6, t.ones_like(norms), norms)
        batch_normed = batch / norms
        return batch_normed

    def calculate_loss(
        self,
        out: Float[Tensor, "batch inst feats"],
        batch: Float[Tensor, "batch inst feats"],
        per_inst: bool = False,
    ) -> Float[Tensor, "inst"]:
        """
        New function to calculate loss, because we need a "loss per instance" option to find the best
        instance at the end of our optimization.
        """
        error = self.importance * ((batch - out) ** 2)
        loss = einops.reduce(error, "batch inst feats -> inst", "mean")
        return loss.sum() if per_inst else loss

    def optimize(
        self,
        batch_size: int,
        num_batch_updates: int = NUM_BATCH_UPDATES,
        log_freq: int = 100,
        lr: float = LEARNING_RATE,
        lr_scale: Callable[[int, int], float] = anthropic_lr,
        weight_decay: float = WEIGHT_DECAY,
    ) -> tuple[Tensor, Tensor]:
        optimizer = t.optim.AdamW(list(self.parameters()), lr=lr, weight_decay=weight_decay)

        progress_bar = tqdm(range(num_batch_updates))

        # Same batch for each step
        batch = self.generate_batch(batch_size)  # [batch_size inst n_features]

        for step in progress_bar:
            # Update learning rate
            step_lr = lr * lr_scale(step, num_batch_updates)
            for group in optimizer.param_groups:
                group["lr"] = step_lr

            # Optimize
            optimizer.zero_grad()
            out = self.forward(batch)
            loss = self.calculate_loss(out, batch)
            loss.backward()
            optimizer.step()

            # Display progress bar
            if (step % log_freq == 0) or (step + 1 == num_batch_updates):
                progress_bar.set_postfix(loss=loss.item() / self.cfg.n_inst, lr=step_lr)

        # Generate one final batch to compute the loss (we want only the best instance!)
        with t.inference_mode():
            out = self.forward(batch)
            loss_per_inst = self.calculate_loss(out, batch, per_inst=True)
            best_inst = loss_per_inst.argmin()
            print(f"Best instance = #{best_inst}, with loss {loss_per_inst[best_inst].item():.4e}")

        return batch[:, best_inst], self.W[best_inst].detach()
```

Now, actually generating the data:

```python
features_list = []
hidden_representations_list = []

for batch_size in tqdm(BATCH_SIZES):
    # Define our model
    cfg = Config(n_features=N_FEATURES, n_inst=N_INST, d_hidden=D_HIDDEN)
    model = DoubleDescentModel(cfg, feature_probability=FEATURE_PROBABILITY).to(device)

    # Optimize, and return the best batch & weight matrix
    batch_inst, W_inst = model.optimize(batch_size=batch_size, num_batch_updates=15_000)

    # Calculate the hidden feature representations, and add both this and weight matrix to our lists of data
    with t.inference_mode():
        hidden = einops.einsum(
            batch_inst, W_inst, "batch features, hidden features -> hidden batch"
        )
    features_list.append(W_inst.cpu())
    hidden_representations_list.append(hidden.cpu())
```

Visualising the 2D feature plots:

```python
utils.plot_features_in_2d(
    features_list + hidden_representations_list,
    colors=[["blue"] for _ in range(len(BATCH_SIZES))] + [["red"] for _ in range(len(BATCH_SIZES))],
    title="Double Descent & Superposition (num features = 1000)",
    subplot_titles=[f"Features (batch={bs})" for bs in BATCH_SIZES] + [f"Data (batch={bs})" for bs in BATCH_SIZES],
    allow_different_limits_across_subplots=True,
    n_rows=2,
)
```

<img src="https://raw.githubusercontent.com/callummcdougall/computational-thread-art/master/example_images/misc/ddd_fig1.png" width="1400">

and the dimensionality of features / data:

```python
df_data = {"Batch size": [], "Dimensionality": [], "Data": []}

for batch_size, model_W, hidden in zip(BATCH_SIZES, features_list, hidden_representations_list):
    # Get x-axis data (batch size), and color (blue or red)
    df_data["Batch size"].extend([batch_size] * (N_FEATURES + batch_size))
    df_data["Data"].extend(["features"] * N_FEATURES + ["hidden"] * batch_size)

    # Calculate dimensionality of model.W[inst].T, which has shape [d_hidden=2 N_FEATURES]
    feature_dim = DoubleDescentModel.dimensionality(model_W.T)
    assert feature_dim.shape == (N_FEATURES,)

    # Calculate dimensionality of model's batch data hidden representation. This has shape [d_hidden=2 batch_size]
    data_dim = DoubleDescentModel.dimensionality(hidden.T)
    assert data_dim.shape == (batch_size,)

    # Add them both to the data
    df_data["Dimensionality"].extend(feature_dim.tolist() + data_dim.tolist())


df = pd.DataFrame(df_data)
eps = 0.01
xline1, xline2 = (100 * 200) ** 0.5, (500 * 1000) ** 0.5
vrect_kwargs: dict[str, Any] = dict(opacity=0.5, layer="below", line_width=0)
xrange = [math.log10(1.5), math.log10(5000)]
fig = (
    px.strip(
        df,
        x="Batch size",
        y="Dimensionality",
        color="Data",
        color_discrete_sequence=["rgba(0,0,255,0.3)", "rgba(255,0,0,0.3)"],
        log_x=True,
        template="simple_white",
        width=1000,
        height=600,
        title="Dimensionality of features & hidden representation of training examples",
    )
    .update_traces(marker=dict(opacity=0.5))
    .update_layout(
        xaxis=dict(range=xrange, tickmode="array", tickvals=BATCH_SIZES),
        yaxis=dict(range=[-0.05, 1.0]),
    )
    .add_vrect(x0=1, x1=(1 - eps) * xline1, fillcolor="#ddd", **vrect_kwargs)
    .add_vrect(x0=(1 + eps) * xline1, x1=(1 - eps) * xline2, fillcolor="#ccc", **vrect_kwargs)
    .add_vrect(x0=(1 + eps) * xline2, x1=10_000, fillcolor="#bbb", **vrect_kwargs)
    .add_scatter(
        x=BATCH_SIZES,
        y=[2 / b for b in BATCH_SIZES],
        mode="lines",
        line=dict(shape="spline", dash="dot", color="#333", width=1),
        name="d_hidden / batch_size",
    )
)

fig.show(config=dict(staticPlot=True))
```

## Suggested topics for further exploration



### [Softmax Linear Units](https://transformer-circuits.pub/2022/solu/index.html)

This is a proposed architectural change which appears to increase the number of interpretable MLPs with low performance cost. In particular, it may reduce the instance of superposition.

TL;DR: SOLU is an activation function $\vec{x} \to \vec{x} * \operatorname{softmax}(\vec{x})$ which encourages sparsity in activations in the same way that softmax encourages sparsity (often softmax'ed probability distributions have one probability close to one and the others close to zero). Encouraging activation sparsity might make it harder for neurons to be polysemantic.

Replication of the results of this paper might not be a practical final week project. However, several transformers in the TransformerLens library have been trained with SOLU (see the [model page](https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html) for more details), which makes them a good candidate for closer study. Some questions you might want to explore:

- How do Neel's SoLU and GELU models compare in [neuroscope](https://neuroscope.io/) under the polysemanticity metric used in the SoLU paper? (what fraction of neurons seem monosemantic when looking at the top 10 activating dataset examples for 1 minute)
- The SoLU metrics for polysemanticity are somewhat limited, since they only provide information about whether a neuron is monosemantic when activating strongly (and this may not be corrrelated to whether it is monosemantic in general - see [this caveat](https://transformer-circuits.pub/2022/solu/index.html#:~:text=be%20reverse%2Dengineered.-,CAVEAT,-Since%20publication%2C%20we%27ve) in the paper). Can you find any better metrics? Can you be more reliable, or more scalable?
- The paper [speculates](https://transformer-circuits.pub/2022/solu/index.html#section-4-3) that the LayerNorm after the SoLU activations lets the model "smuggle through" superposition, by smearing features across many dimensions, having the output be very small, and letting the LayerNorm scale it up. Can you find any evidence of this in solu-1l?



### [Towards Monosemanticity: Decomposing Language Models With Dictionary Learning](https://transformer-circuits.pub/2023/monosemantic-features/index.html)

There are many other interesting topics from Anthropic's dictionary learning paper which we didn't have time to dive into here, such as automated interpretability, feature motifs, and finite-state automata.

There's also a [Future Work](https://transformer-circuits.pub/2023/monosemantic-features/index.html#discussion-future-work) section at the end, which readers might find interesting for any project ideas!





### [Exciting Open Problems In Mech Interp v2](https://docs.google.com/document/d/1lIIzMjenXh-U0j5jkuqSDTawCoMNW4TqUlxk7mmbmRg/edit)

This document was written by Neel, and it collates a bunch of interesting open problems in mech interp (with a strong focus on SAE-related ones). Again, many of these could make great capstone projects! We encourage you to pick more achievable, less ambitious projects from this list though.

If any of the projects you're interested in involve training a sparse autoencoder, we *strongly* recommend [this post](https://www.lesswrong.com/posts/fifPCos6ddsmJYahD/my-best-guess-at-the-important-tricks-for-training-1l-saes) by Arthur Conmy, which collates a bunch of different techniques for training SAEs well (most of which we didn't cover in these exercises).